## Initialisation

In [ ]:
import glob2
import pickle
from pathlib import Path
import os


from scipy import stats

import multiprocessing as mp
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.interpolate import griddata
from scipy.stats import chi2


import networkx as nx
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import pandas as pd
from datetime import datetime,timedelta, timezone

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import matplotlib as mpl

from utils.data_reading.sound_data.station import StationsCatalog
from src.utils.simulation.synthetic import RealStationDataGenerator
# from src.utils.physics.sound_model.spherical_sound_model import HomogeneousSphericalSoundModel as HomogeneousSoundModel
from src.utils.physics.sound_model.ellipsoidal_sound_model import HomogeneousEllipsoidalSoundModel as HomogeneousSoundModel
from src.utils.physics.sound_model.ellipsoidal_sound_model import GridEllipsoidalSoundModel as HomogeneousSoundModel

from utils.detection.association_geodesic_ridges import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids
mpl.rcParams.update({
    "font.size": 12,
    "axes.titlesize": 12,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "figure.titlesize": 12,
    "font.family": "serif",
    "pdf.fonttype": 42,
    "ps.fonttype": 42
})

In [ ]:
catalog_path = "/media/rsafran/CORSAIR/OHASISBIO/recensement_stations_OHASISBIO_RS.csv"
dataset ="OHASISBIO-2018"
STATION = StationsCatalog(catalog_path).by_dataset(dataset)
# SOUND_MODEL = HomogeneousSoundModel(sound_speed=1485.5)
ISAS_PATH = "/media/rsafran/CORSAIR/ISAS/extracted/2018"
arr = os.listdir(ISAS_PATH)
file_list = [os.path.join(ISAS_PATH, fname) for fname in arr if fname.endswith('.nc')]
SOUND_MODEL = HomogeneousSoundModel(file_list)

In [ ]:
for st in STATION :
    print(st.get_pos())

In [ ]:
gen = RealStationDataGenerator(STATION, SOUND_MODEL)
data = gen.generate_events(datetime(year=2018, month=1, day=1))
gen.plot_stations_and_events()

In [ ]:
[lat_min, lat_max] = [-60, 5]
[lon_min, lon_max] = [35, 120]
LAT_BOUNDS = [-60, 5]
LON_BOUNDS = [35, 120]


### Creation

In [ ]:
# Example usage with enhanced features

# Create generator with ridge-based events and clock drift
generator = RealStationDataGenerator(
    stations=STATION,
    sound_model=SOUND_MODEL,
    n_real_events=100,
    n_noise_detections=450,
    ridge_data_path="../../../data/dorsales/",
    events_bounds=np.array([[lat_min,0*lat_max-1], [lon_min, lon_max]]),
    ridge_std_km=100,  # Events within 100km of ridges
    perfect_events=False,  # Add timing noise
    apply_clock_drift=True,  # Apply clock drift errors
    reference_time_years=1,  # 2 years reference time
    seed=42
)

# Generate events
start_time = datetime(2023, 1, 1)
events, ground_truth = generator.generate_events(start_time, duration_hours=2*24) #100 en 48h c'est assez réaliste

# Plot results
generator.plot_stations_and_events()

# Show simulation info
print("Simulation parameters:")
for key, value in generator.get_simulation_info().items():
    print(f"  {key}: {value}")

In [ ]:
NEAMS = STATION.by_name("NEAMS").stations[0]
NEAMS.get_clock_error(datetime(year=2019, month=2, day=5))
# NEAMS.date_start
n_real_events = generator.get_simulation_info()["n_real_events"]

In [ ]:
events.station[0].other_kwargs

In [ ]:
pos_diff= []
time_diff = []
pos_events = []
quality = []
all_residus = []

chi2_stats = []
p_values = []
passes = []
outliers = []

def analyze_residuals(result):
    """Analyse la qualité des résidus pour évaluer la pertinence de l'association"""
    residuals = result.fun
    num_params = 2
    num_obs = len(residuals)
    dof = max(1, num_obs - num_params)

    if num_obs == 0:
        print("error")

    # Chi-carré réduit
    chi2_reduced = np.sum(residuals**2) / dof

    # Z-score des résidus (résidus standardisés)
    residual_std = np.std(residuals) if np.std(residuals) > 1e-10 else 1e-10
    z_scores = np.abs(residuals) / residual_std
    max_zscore = np.max(z_scores)

    # Fraction d'outliers (|z-score| > 2.5)
    outlier_fraction = np.mean(z_scores > 2.0)

    # Test unitaire de variance
    p = 2  # nombre de paramètres ajustés (ici a et b)
    sigma_theorique = 3
    s2 = np.var(residuals, ddof=p)  # variance sans biais

    # --- 5. Test unitaire de variance ---
    chi2_stat = (num_obs - p) * s2 / sigma_theorique**2

    # Intervalle de confiance au seuil 95%
    alpha = 0.05
    chi2_inf = chi2.ppf(alpha / 2, df=num_obs - p)
    chi2_sup = chi2.ppf(1 - alpha / 2, df=num_obs - p)
    if  chi2_inf < chi2_stat < chi2_sup:
        H0 = chi2_stat
        #print("✅ On ne rejette pas H0 : la variance est compatible avec σ² théorique")
    else:
        H0 = -chi2_stat
        #print("❌ On rejette H0 : la variance diffère significativement de σ² théorique")

    # Test de normalité des résidus (Shapiro-Wilk si n < 5000, sinon Kolmogorov-Smirnov)
    if num_obs >= 3 and num_obs < 5000:
         _, normality_pvalue = stats.shapiro(residuals)

    # Score de qualité composite (plus c'est haut, mieux c'est)
    quality_score = 1.0 / (1.0 + chi2_reduced)  # Pénalise chi2 élevé
    quality_score *= (1.0 - outlier_fraction)    # Pénalise les outliers
    quality_score *= 1.0 / (1.0 + max_zscore/10.0)  # Pénalise les résidus extrêmes
    quality_score+=H0

    return {
        'chi2_reduced': float(chi2_reduced),
        'max_residual_zscore': float(max_zscore),
        'outlier_fraction': float(outlier_fraction),
        'normality_pvalue': float(normality_pvalue),
        'quality_score': float(quality_score),
        'cost':float(res.cost),
        'H0': float(H0)
    }

for i in range(n_real_events):

    event_extracted = events[9*i:9*(1+i)]
    pos = [e.get_pos() for e in event_extracted.station]
    det = [e for e in event_extracted.datetime]
    time_elapsed_seconds = 365.25*24*3600*2
    drift = [e.other_kwargs["clock_drift_ppm"] * 1e-6 * time_elapsed_seconds  if e.other_kwargs['gps_sync'] != 'ok' else 0 for e in event_extracted.station]
    drift = np.abs(drift) / np.sqrt(3)
    min_date = np.argmin(det)
    c0 = [-8.,68.]
    t0 = -1 * SOUND_MODEL.get_sound_travel_time(c0, pos[min_date], det[min_date])
    pick_uncertainties = [3]*len(det)

    x0 = [t0]+list(c0)
    x0 = np.array(x0)
    # print(x0)
    res= SOUND_MODEL.localize_with_uncertainties(
        pos, det, initial_pos=x0, drift_uncertainties = drift, pick_uncertainties = pick_uncertainties, x_min=lat_min-1, x_max=lat_max+1,y_min=lon_min-1,y_max=lon_max+1
    )
    out = SOUND_MODEL.detect_outliers(res)
        # Test chi²
    chi2_stat, p_val, pass_test, dof = SOUND_MODEL.test_chi_square(res)
    chi2_stats.append(chi2_stat)
    p_values.append(p_val)
    passes.append(pass_test)
    outliers.append(out)



    all_residus.append(res)
    quality.append(analyze_residuals(res))
    # print(res.x[1:], datetime.datetime.utcfromtimestamp(res.x[0]))
    err = res.x[1:] - ground_truth.iloc[i][['lat','lon']].values
    t_err = (ground_truth.iloc[i].origin_time.tz_localize("UTC")-datetime.fromtimestamp(res.x[0], tz=timezone.utc)).total_seconds()
    pos_ev = res.x[1:]
    pos_events.append(pos_ev)
    pos_diff.append(err)
    time_diff.append(t_err)


# Analyse
print(f"Taux de réussite: {np.mean(passes)*100:.1f}%")
print(f"Chi² moyen: {np.mean(chi2_stats):.2f} (attendu: ~{dof})")
print(f"Chi²/dof moyen: {np.mean(chi2_stats)/dof:.2f} (attendu: ~1)")
quality  = pd.DataFrame(quality)
pos_events = np.array(pos_events)

plt.subplot(311)
plt.subplots_adjust(left=3, right=4.0)
plt.hist(np.array(pos_diff)[:,1],alpha=0.5, label='diff lon (°)', bins=50, density=True)
plt.hist(np.array(pos_diff)[:,0],alpha=0.5, label='diff lat (°)', bins=50, density=True)
plt.legend()
plt.subplot(312)
plt.subplots_adjust(left=3, right=4.0)
plt.hist(np.array(time_diff), label='time diff', bins=50, density=True)
plt.legend()
plt.subplot(313)
plt.subplots_adjust(left=3, right=4.0)
plt.hist(quality['chi2_reduced'], label='chi2_reduced', bins=50, density=True)
plt.legend()

In [ ]:
# FONCTION DE DIAGNOSTIC
def diagnose_localization_quality(events, ground_truth, n_real_events, pick_uncertainties_sigma=3):
    """
    Diagnostic complet pour identifier le problème de p-value
    """
    chi2_stats = []
    p_values = []
    passes = []
    chi2_reduced = []
    position_errors = []
    time_errors = []
    outliers = []
    for i in range(n_real_events):
        event_extracted = events[9*i:9*(1+i)]
        pos = [e.get_pos() for e in event_extracted.station]
        det = [e for e in event_extracted.datetime]
        min_date = np.argmin(det)
        c0 = [-8., 68.]
        t0 = -1 * SOUND_MODEL.get_sound_travel_time(c0, pos[min_date], det[min_date])
        pick_uncertainties = [pick_uncertainties_sigma] * len(det)
        x0 = [t0] + list(c0)
        (lat_max, lat_max, lon_min, lon_max) = (-69.85174, 19.605793, 25.0, 130.0)
        res = SOUND_MODEL.localize_with_uncertainties(
            pos, det, initial_pos=x0, pick_uncertainties=pick_uncertainties, x_min=lat_min, x_max=lat_max,y_min=lon_min,y_max=lon_max
        )
        out = SOUND_MODEL.detect_outliers(res)
        outliers.append(out)
        # Erreurs de position
        err = res.x[1:] - ground_truth.iloc[i][['lat','lon']].values
        position_errors.append(np.linalg.norm(err))

        # Erreur de temps
        t_err = (ground_truth.iloc[i].origin_time.tz_localize("UTC")-datetime.fromtimestamp(res.x[0], tz=timezone.utc)).total_seconds()
        time_errors.append(t_err)

        # Test chi²
        chi2_stat, p_val, pass_test, dof = SOUND_MODEL.test_chi_square(res)
        chi2_stats.append(chi2_stat)
        p_values.append(p_val)
        passes.append(pass_test)
        chi2_reduced.append(chi2_stat / dof)

    # Résultats
    print("=" * 60)
    print("DIAGNOSTIC DE QUALITÉ DE LA LOCALISATION")
    print("=" * 60)
    print(f"\n1. TEST CHI² (α=0.05)")
    print(f"   Taux de réussite: {np.mean(passes)*100:.1f}% (attendu: 95%)")
    print(f"   DOF: {dof}")
    print(f"   Chi² moyen: {np.mean(chi2_stats):.2f} (attendu: ~{dof})")
    print(f"   Chi²/DOF moyen: {np.mean(chi2_reduced):.3f} (attendu: ~1.0)")

    print(f"\n2. INTERPRÉTATION")
    if np.mean(chi2_reduced) > 1.5:
        print(f"   ⚠️  Chi²/DOF > 1.5: Incertitudes SOUS-ESTIMÉES")
        print(f"   → Les résidus sont trop grands par rapport aux incertitudes")
        print(f"   → Solutions: augmenter pick_uncertainties ou vérifier le modèle")
    elif np.mean(chi2_reduced) < 0.5:
        print(f"   ⚠️  Chi²/DOF < 0.5: Incertitudes SUR-ESTIMÉES")
        print(f"   → Les résidus sont trop petits par rapport aux incertitudes")
        print(f"   → Solutions: diminuer pick_uncertainties")
    else:
        print(f"   ✓ Chi²/DOF proche de 1: Incertitudes bien calibrées")

    print(f"\n3. ERREURS DE POSITION")
    print(f"   Erreur moyenne: {np.mean(position_errors):.3f}°")
    print(f"   Erreur médiane: {np.median(position_errors):.3f}°")
    print(f"   Erreur max: {np.max(position_errors):.3f}°")

    print(f"\n4. ERREURS DE TEMPS")
    print(f"   Erreur moyenne: {np.mean(np.abs(time_errors)):.3f} s")
    print(f"   Erreur médiane: {np.median(np.abs(time_errors)):.3f} s")

    print(f"\n5. P-VALUES")
    print(f"   P-value médiane: {np.median(p_values):.3f}")
    print(f"   P-value < 0.05: {np.sum(np.array(p_values) < 0.05)} événements")

    return {
        'chi2_stats': chi2_stats,
        'p_values': p_values,
        'passes': passes,
        'chi2_reduced': chi2_reduced,
        'position_errors': position_errors,
        'time_errors': time_errors
    }

results = diagnose_localization_quality(events, ground_truth, n_real_events, pick_uncertainties_sigma=3)

# Pour visualiser la distribution des p-values
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(131)
plt.hist(results['p_values'], bins=20, edgecolor='black')
plt.axvline(0.05, color='red', linestyle='--', label='α=0.05')
plt.xlabel('P-value')
plt.ylabel('Fréquence')
plt.title('Distribution des p-values')
plt.legend()

plt.subplot(132)
plt.hist(results['chi2_reduced'], bins=20, edgecolor='black')
plt.axvline(1.0, color='red', linestyle='--', label='Attendu')
plt.xlabel('Chi²/DOF')
plt.ylabel('Fréquence')
plt.title('Chi² réduit')
plt.legend()

plt.subplot(133)
plt.scatter(results['chi2_reduced'], results['p_values'])
plt.axhline(0.05, color='red', linestyle='--')
plt.axvline(1.0, color='red', linestyle='--')
plt.xlabel('Chi²/DOF')
plt.ylabel('P-value')
plt.title('Chi² vs P-value')

plt.tight_layout()
plt.show()

In [ ]:
n = 48#len(np.array(time_diff)[abs(np.array(time_diff)) > 180])+len(np.array(pos_diff)[:,1][abs(np.array(pos_diff)[:,1]) > 1])+len(np.array(pos_diff)[:,0][abs(np.array(pos_diff)[:,1]) > 1])
p = (1-n/500)*100
print(n,p)

In [ ]:
pd.DataFrame(outliers)['outlier_indices'].apply(lambda x : len(x))

In [ ]:
plt.figure(figsize=(8,8 ),dpi=300)
pos_diff = np.array(pos_diff)
tot_pos_diff =pos_diff[:,1]
ax1=  plt.subplot(211)
plt.scatter(pos_events[:,1], pos_events[:,0], c=time_diff, alpha=0.85,vmin=40,vmax=-40 ,cmap='seismic')
plt.colorbar()
ax2= plt.subplot(223)
plt.scatter(pos_events[:,1], pos_events[:,0],c=pos_diff[:,1], alpha=0.85, cmap='seismic',vmin=-0.51, vmax=0.51, label='lon diff')
plt.tight_layout()
plt.legend()
ax3 = plt.subplot(224,sharex=ax2,sharey=ax2)
# make these tick labels invisible
plt.tick_params('y', labelleft=False)
plt.scatter(pos_events[:,1], pos_events[:,0],c=pos_diff[:,0], alpha=0.85, cmap='seismic',vmin=-0.51, vmax=0.51, label='lat diff')
plt.colorbar()
plt.legend()
plt.tight_layout()

In [ ]:
print((np.quantile(pos_diff[:,1],0.99)))
print(np.quantile(pos_diff[:,0],0.99))
print(np.quantile(time_diff,0.99))

In [ ]:
len(pos_diff[:,0]**2 + pos_diff[:,1]**2)

In [ ]:
def process_synthetic_detections(events_df,
                                min_p_tissnet_primary=0.1,
                                min_p_tissnet_secondary=0.1,
                                merge_delta=timedelta(seconds=5)):
    """
    Process synthetic detections from the RealStationDataGenerator

    Parameters:
    -----------
    events_df : DataFrame
        With columns ['datetime', 'station', 'probability', 'true_event']
    min_p_tissnet_primary : float
        Minimum probability threshold for primary detections
    min_p_tissnet_secondary : float
        Minimum probability threshold for secondary detections
    merge_delta : timedelta
        Time delta for merging close detections

    Returns:
    --------
    detections : dict
        station -> np.array([[datetime, probability], ...])
    detections_merged : np.ndarray
        Array of [datetime, probability, station], sorted by datetime
    """

    # Garder uniquement les événements qui passent le seuil secondaire
    filtered_events = events_df[events_df['probability'] > min_p_tissnet_secondary].copy()

    detections = {}
    detections_merged_list = []

    # Groupement par station
    for station in filtered_events['station'].unique():
        station_events = filtered_events[filtered_events['station'] == station].copy()
        station_events = station_events.sort_values('datetime')

        d = station_events[['datetime', 'probability']].values

        if len(d) == 0:
            detections[station] = np.array([]).reshape(0, 2)
            continue

        # Nettoyage des détections proches ou régulières
        new_d = [d[0]]
        for i in range(1, len(d)):
            dt = (d[i, 0] - d[i - 1, 0]).total_seconds()
            if dt > merge_delta.total_seconds():
                if i < 3:
                    new_d.append(d[i])
                else:
                    dt1 = (d[i, 0] - d[i - 1, 0]).total_seconds()
                    dt2 = (d[i - 1, 0] - d[i - 2, 0]).total_seconds()

                    condition1 = abs(dt1 - dt2) > merge_delta.total_seconds()

                    if i >= 4:
                        dt3 = (d[i - 1, 0] - d[i - 3, 0]).total_seconds()
                        dt4 = (d[i - 2, 0] - d[i - 4, 0]).total_seconds()
                        condition2 = abs(dt3 - dt4) > merge_delta.total_seconds()
                    else:
                        condition2 = True

                    if condition1 and condition2:
                        new_d.append(d[i])

        d = np.array(new_d, dtype=object)
        detections[station] = d

        print(f"Found {len(d)} detections for station {station}")

        # Ajouter à la liste globale
        for det in d:
            detections_merged_list.append([det[0], det[1], station])

    # Création du tableau final
    if len(detections_merged_list) == 0:
        detections_merged = np.array([]).reshape(0, 3)
    else:
        detections_merged = np.array(detections_merged_list, dtype=object)

        # Filtrer sur le seuil primaire
        detections_merged = detections_merged[detections_merged[:, 1] > min_p_tissnet_primary]

        # Trier par datetime
        if len(detections_merged) > 0:
            detections_merged = detections_merged[np.argsort(detections_merged[:, 0])]

    return detections, detections_merged


def analyze_synthetic_detections(detections, detections_merged, ground_truth_df=None):
    """
    Analyze the processed synthetic detections
    """
    analysis = {}

    # Basic statistics
    total_detections = sum(len(det) for det in detections.values())
    analysis['total_detections'] = total_detections
    analysis['total_merged_detections'] = len(detections_merged)
    analysis['num_stations_with_detections'] = len([s for s in detections if len(detections[s]) > 0])

    # Per station statistics
    station_stats = {}
    for station, dets in detections.items():
        station_stats[str(station)] = {
            'num_detections': len(dets),
            'avg_probability': np.mean(dets[:, 1]) if len(dets) > 0 else 0,
            'std_probability': np.std(dets[:, 1]) if len(dets) > 0 else 0
        }
    analysis['station_stats'] = station_stats

    # Time span analysis
    if len(detections_merged) > 1:
        start_time = detections_merged[0, 0]
        end_time = detections_merged[-1, 0]
        time_span_seconds = (end_time - start_time).total_seconds()
        analysis['time_span_hours'] = time_span_seconds / 3600
        analysis['detection_rate_per_hour'] = len(detections_merged) / (time_span_seconds / 3600)

    # Ground truth comparison if available
    if ground_truth_df is not None:
        analysis['num_true_events'] = len(ground_truth_df)
        analysis['detection_efficiency'] = total_detections / len(ground_truth_df) if len(ground_truth_df) > 0 else 0

    return analysis



def plot_detection_timeline(detections_merged, ground_truth_df=None):
    """
    Plot timeline of detections and optionally ground truth events
    """


    if len(detections_merged) == 0:
        print("No detections to plot")
        return

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8), sharex=True)

    # Use datetime directly
    timestamps = detections_merged[:, 0]
    probabilities = detections_merged[:, 1].astype(float)

    # Plot detections timeline
    ax1.scatter(timestamps, probabilities, alpha=0.6, s=30)
    ax1.set_ylabel('Detection Probability')
    ax1.set_title('Synthetic Detections Timeline')
    ax1.grid(True, alpha=0.3)

    # Plot detections per station
    station_objects = detections_merged[:, 2]
    station_strings = [str(s) for s in station_objects]
    unique_station_strings = list(set(station_strings))

    # Mapping string -> original station object
    station_map = {}
    for i, station_str in enumerate(station_strings):
        if station_str not in station_map:
            station_map[station_str] = station_objects[i]

    unique_stations = [station_map[s] for s in unique_station_strings]
    station_colors = plt.cm.tab10(np.linspace(0, 1, len(unique_stations)))

    for i, station in enumerate(unique_stations):
        station_mask = np.array([s == station for s in station_objects])
        station_times = np.array(timestamps)[station_mask]
        ax2.scatter(station_times, [i] * len(station_times),
                   c=[station_colors[i]], alpha=0.7, s=50,
                   label=str(station))

    ax2.set_ylabel('Station')
    ax2.set_xlabel('Time')
    ax2.set_title('Detections by Station')
    ax2.set_yticks(range(len(unique_stations)))
    ax2.set_yticklabels([str(s) for s in unique_stations])
    ax2.grid(True, alpha=0.3)

    # Format x-axis
    ax2.xaxis.set_major_formatter(DateFormatter('%H:%M'))

    # Add ground truth if available
    if ground_truth_df is not None:
        for idx, event in ground_truth_df.iterrows():
            ax1.axvline(event['origin_time'], color='red', alpha=0.5, linestyle='--')
            ax2.axvline(event['origin_time'], color='red', alpha=0.5, linestyle='--')

    plt.tight_layout()
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()



# Exemple d'utilisation
if __name__ == "__main__":

    # Traiter les détections synthétiques
    DETECTIONS, DETECTIONS_MERGED = process_synthetic_detections(
        events,
        min_p_tissnet_primary=0.0,
        min_p_tissnet_secondary=0.0,
        merge_delta=timedelta(seconds=0)
    )
    #
    # # Analyser les résultats
    # analysis = analyze_synthetic_detections(DETECTIONS, DETECTIONS_MERGED, ground_truth)
    #
    # print("\nDetection Analysis:")
    # for key, value in analysis.items():
    #     if key != 'station_stats':
    #         print(f"{key}: {value}")
    #
    # # Visualiser la timeline
    # plot_detection_timeline(DETECTIONS_MERGED, ground_truth)

## Association

In [ ]:

MIN_P_TISSNET_PRIMARY = 0.1
STATIONS = [s for s in DETECTIONS.keys()]
FIRSTS_DETECTIONS = {s : DETECTIONS[s][0,0] for s in STATIONS}
LASTS_DETECTIONS = {s : DETECTIONS[s][-1,0] for s in STATIONS}

# do not keep detection entries for which the detection list is empty
to_del = []
for s in DETECTIONS.keys():
    if len(DETECTIONS[s]) == 0:
        to_del.append(s)
for s in to_del:
    del DETECTIONS[s]

# assign an index to each detection
idx_det = 0
IDX_TO_DET = {}
for idx, s in enumerate(DETECTIONS.keys()):
    s.idx = idx  # indexes to store efficiently the associations
    DETECTIONS[s] = list(DETECTIONS[s])
    for i in range(len(DETECTIONS[s])):
        DETECTIONS[s][i] = np.concatenate((DETECTIONS[s][i], [idx_det]))
        IDX_TO_DET[idx_det] = DETECTIONS[s][i]
        idx_det += 1
    DETECTIONS[s] = np.array(DETECTIONS[s])
DETECTION_IDXS = np.array(list(range(idx_det)))

# only keep the stations that appear in the kept detections
STATIONS = [s for s in DETECTIONS.keys()]
FIRSTS_DETECTIONS = {s : DETECTIONS[s][0,0] for s in STATIONS}
LASTS_DETECTIONS = {s : DETECTIONS[s][-1,0] for s in STATIONS}

# list that will be browsed
DETECTIONS_MERGED = np.concatenate([[(det[0], det[1], det[2], s) for det in DETECTIONS[s]] for s in STATIONS if "IMS" not in s.dataset])
DETECTIONS_MERGED = DETECTIONS_MERGED[DETECTIONS_MERGED[:, 1] > MIN_P_TISSNET_PRIMARY]
DETECTIONS_MERGED = DETECTIONS_MERGED[np.argsort(DETECTIONS_MERGED[:, 1])][::-1]

In [ ]:
ASSOCIATION_OUTPUT_DIR = "../../../data"
DETECTIONS_DIR_NAME = "synthetic_detections"

MIN_P_TISSNET_PRIMARY = 0.1
MIN_P_TISSNET_SECONDARY = 0.0
MERGE_DELTA_S = 0 # threshold below which we consider two events should be merged
REQ_CLOSEST_STATIONS = 4  # The REQ_CLOSEST_STATIONS th closest stations will be required for an association to be valid
RUN_ASSOCIATION = True # set to False to load previous associations without processing it again
NCPUS = 24
SAVE_PATH_ROOT = None

In [ ]:
with open("../../../data/T-pick/grid_to_coords_ridges.pkl", "rb") as f:
    GRID_TO_COORDS = pickle.load(f)

In [ ]:
PICK_UNCERTAINTY = 2
SOUND_SPEED_UNCERTAINTY = 6
MAX_CLOCK_DRIFT = 0

GRID_PATH = f"{ASSOCIATION_OUTPUT_DIR}/synthetic_detections/grids_{PICK_UNCERTAINTY}_{SOUND_SPEED_UNCERTAINTY}_{MAX_CLOCK_DRIFT}.pkl"

if not Path(GRID_PATH).exists():
    GRID_TO_COORDS, TDoA, MAX_TDoA, TDoA_UNCERTAINTIES, travel_times, travel_time_uncertainties = compute_grids(GRID_TO_COORDS, SOUND_MODEL, STATIONS, pick_uncertainty=PICK_UNCERTAINTY, sound_speed_uncertainty=SOUND_SPEED_UNCERTAINTY, max_clock_drift=MAX_CLOCK_DRIFT)
    with open(GRID_PATH, "wb") as f:
        pickle.dump((GRID_TO_COORDS, TDoA, MAX_TDoA, TDoA_UNCERTAINTIES), f)
else:
    with open(GRID_PATH, "rb") as f:
        GRID_TO_COORDS, TDoA, MAX_TDoA, TDoA_UNCERTAINTIES = pickle.load(f)

In [ ]:


print("starting association")

MIN_ASSOCIATION_SIZE = 5
max_reached_per_det = {det_idx:MIN_ASSOCIATION_SIZE for det_idx in DETECTION_IDXS}

already_examined = set()
associations = []
def process_detection(arg):
    detection, already_examined, max_reached_per_det = arg
    local_association = []
    date1, p1, idx_det1, s1 = detection
    save_path = SAVE_PATH_ROOT
    if save_path is not None:
        save_path = f'{save_path}/{s1.name}-{date1.strftime("%Y%m%d_%H%M%S")}'
        Path(save_path).mkdir(parents=True, exist_ok=True)

    # list all other stations and sort them by distance from s1
    other_stations = np.array([s2 for s2 in STATIONS if s2 != s1
                               and date1 + timedelta(days=1) > FIRSTS_DETECTIONS[s2]
                               and date1 - timedelta(days=1) < LASTS_DETECTIONS[s2]])
    other_stations = other_stations[np.argsort([MAX_TDoA[s1][s2][date1.month - 1] for s2 in other_stations])]

    # given the detection date1 occurred on station s1, list all the detections of other stations that may be generated by the same source event
    current_association = {s1:(date1, idx_det1)}
    candidates = compute_candidates(other_stations, current_association, DETECTIONS, MAX_TDoA, MERGE_DELTA_S)

    # update the list of other stations to only include the ones having at least a candidate detection
    other_stations = [s for s in other_stations if len(candidates[s]) > 0]

    # define the recursive browsing function (that is responsible for browsing the search space of associations for s1-date1)
    def backtrack(station_index, current_association, valid_grid, associations, save_path):
        if station_index == len(other_stations):
            return
        station = other_stations[station_index]

        candidates = compute_candidates([station], current_association, DETECTIONS, MAX_TDoA, MERGE_DELTA_S)
        for idx in candidates[station]:
            date, p, idx_det = DETECTIONS[station][idx]

            if date in already_examined:
                # the det was already browsed as main
                continue
            if len(other_stations) <= max_reached_per_det[idx_det]-1:
                # the det already belongs to an association larger that what we could have here
                continue

            valid_grid_new, dg_new = update_valid_grid(current_association, valid_grid, station, date, TDoA, TDoA_UNCERTAINTIES)

            valid_points_new = np.argwhere(valid_grid_new)[:,0]

            if len(valid_points_new) > 0:
                current_association[station] = (date, idx_det)

                backtrack(station_index + 1, current_association, valid_grid_new, associations, save_path)

                if np.all([len(current_association) >= max_reached_per_det[idx]-1 for _, idx in current_association.values()]):
                    update_results(date1, current_association, valid_points_new, local_association, TDoA, TDoA_UNCERTAINTIES)
                    for _, idx in current_association.values():
                        max_reached_per_det[idx] = max(max_reached_per_det[idx], len(current_association))

                del current_association[station]
        # also try without self
        if station_index >= REQ_CLOSEST_STATIONS:
            max_len_possible = len(current_association) + (len(other_stations) - (station_index+1))
            if np.all([max_len_possible >= max_reached_per_det[idx]-1 for _, idx in current_association.values()]):
                backtrack(station_index + 1, current_association, valid_grid, associations, save_path)
        return

    if len(other_stations) > max_reached_per_det[idx_det1]-1:
        backtrack(0, current_association, None, associations, save_path=save_path)
    return local_association



def process_chunk_simple(chunk_data):
    chunk_detections, max_reached_per_det = chunk_data
    chunk_associations = []
    chunk_examined = set()

    for det in chunk_detections:
        local_association = process_detection((det, chunk_examined, max_reached_per_det))
        chunk_examined.add(det[0])
        chunk_associations.extend(local_association)

    return chunk_associations, chunk_examined

# Diviser en chunks
n_processes = cpu_count()
chunk_size = 50 #len(DETECTIONS_MERGED) // n_processes //20
print(chunk_size)
chunks = [DETECTIONS_MERGED[i:i + chunk_size]
          for i in range(0, len(DETECTIONS_MERGED), chunk_size)]

# Préparer les arguments
chunk_args = [(chunk, max_reached_per_det) for chunk in chunks]

associations = []
already_examined = set()

with ProcessPoolExecutor(max_workers=n_processes) as executor:
    # Soumettre toutes les tâches
    future_to_chunk = {executor.submit(process_chunk_simple, args): i
                      for i, args in enumerate(chunk_args)}

    # Barre de progression avec informations détaillées
    completed_chunks = 0
    processed_detections = 0

    with tqdm(total=len(DETECTIONS_MERGED),
              desc="Processing detections",
              unit="det") as pbar:

        for future in as_completed(future_to_chunk):
            chunk_id = future_to_chunk[future]
            chunk_size_actual = len(chunks[chunk_id])

            try:
                chunk_associations, chunk_examined = future.result()
                associations.extend(chunk_associations)
                already_examined.update(chunk_examined)

                # Mettre à jour la progression
                completed_chunks += 1
                pbar.update(chunk_size_actual)
                pbar.set_postfix({
                    'chunks': f"{completed_chunks}/{len(chunks)}",
                    'associations': len(associations)
                })

            except Exception as exc:
                print(f'Chunk {chunk_id} generated an exception: {exc}')

print(f"Traitement terminé: {len(associations)} associations trouvées")

with open(f"{ASSOCIATION_OUTPUT_DIR}/synthetic_detections/cache/associations.pkl", "wb") as f:
    pickle.dump(associations, f)

## show results map

In [ ]:
local_nb_per_coord = {n: {i:0 for i in range(len(GRID_TO_COORDS))} for n in range(3, 15)}
for j in range(len(associations)):
    if len(associations[j]) != 2 :
        print(len(associations[j]))
    detections, valid_points = associations[j]
    for i in valid_points:
        local_nb_per_coord[len(detections)].setdefault(i, 0)
        local_nb_per_coord[len(detections)][i] += 1
nb_per_coord = local_nb_per_coord
lons = np.array(GRID_TO_COORDS)[:, 1]
lats = np.array(GRID_TO_COORDS)[:, 0]


In [ ]:

# to pass a 1D grid into 2D grid with interpolation
def squarize(coordinates, weights, lat_bounds, lon_bounds, size=1000):
    grid_lat = np.linspace(lat_bounds[0], lat_bounds[-1], size)
    grid_lon = np.linspace(lon_bounds[0], lon_bounds[-1], size)
    grid_lon2d, grid_lat2d = np.meshgrid(grid_lon, grid_lat)

    # Interpolation des poids sur une grille régulière
    grid = griddata(
        coordinates, weights, (grid_lat2d, grid_lon2d),
        method='nearest'
    )

    return grid, grid_lat, grid_lon

plt.rcParams.update({'font.size': 12})

min_size_display = 3
log = True
weights = np.array([np.sum([nb_per_coord[n][i] for n in range(min_size_display,15)]) for i in range(len(GRID_TO_COORDS))])

fig, ax = plt.subplots()
width_in = 6.8
height_in = width_in/1.11
fig.set_size_inches(width_in, height_in)
sq, _, _ = squarize(GRID_TO_COORDS, weights, LAT_BOUNDS, LON_BOUNDS, size=1400)
# if log:
#     sq[sq<1] = 1
#     sq = np.log10(sq)
#     sq[sq>3.5] = 3.5
#     sq[sq<2] = 2
# else:
#     sq[sq>2000] = 2000

im = ax.imshow(sq[::-1], cmap="inferno",extent=(LON_BOUNDS[0], LON_BOUNDS[-1], LAT_BOUNDS[0], LAT_BOUNDS[-1]), vmax=50)
xticks = np.arange(np.floor(LON_BOUNDS[0]/5)*5, np.ceil(LON_BOUNDS[-1]/5)*5 + 5, 5)
yticks = np.arange(np.floor(LAT_BOUNDS[0]/5)*5, np.ceil(LAT_BOUNDS[-1]/5)*5 + 5, 5)
ax.set_xticks(xticks)
ax.set_yticks(yticks)

for s_ in STATIONS:
    p = s_.get_pos()
    if p[0] > LAT_BOUNDS[1] or p[0] < LAT_BOUNDS[0] or p[1] > LON_BOUNDS[1] or p[1] < LON_BOUNDS[0]:
        print(f"Station {s_.name} out of bounds")
        continue
    ax.plot(p[1], p[0], 'yx', alpha=0.75, markersize=10, markeredgewidth=1)
    ax.annotate(s_.name, xy=(p[1], p[0]), xytext=(p[1]-(LON_BOUNDS[1]-LON_BOUNDS[0])/30, p[0]+(LAT_BOUNDS[1]-LAT_BOUNDS[0])/50), textcoords="data", color='y', alpha=0.9, weight='bold')
cbar = plt.colorbar(im,fraction=0.0415, pad=0.04)
cbar.set_label(f'Counts of resulting associations{" (log)" if log else ""}', rotation=270, labelpad=20)
ax.set_title(f"Resulting map")
ax.set_xlabel("lon (°)")
ax.set_ylabel("lat (°)")



In [ ]:
np.max(weights)

In [ ]:
plt.figure()

projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': projection})

for s_ in STATIONS:
    p = s_.get_pos()
    # if p[0] > LAT_BOUNDS[1] or p[0] < LAT_BOUNDS[0] or p[1] > LON_BOUNDS[1] or p[1] < LON_BOUNDS[0]:
    #     print(f"Station {s_.name} out of bounds")
    #     continue
    ax.plot(p[1], p[0], 'yx', alpha=0.75, markersize=10, markeredgewidth=2)
    ax.annotate(s_.name, xy=(p[1], p[0]), xytext=(p[1]-(LON_BOUNDS[1]-LON_BOUNDS[0])/30, p[0]+(LAT_BOUNDS[1]-LAT_BOUNDS[0])/50), textcoords="data", color='y', alpha=0.9, weight='bold')


# COAST

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='gray')
ax.add_feature(cfeature.OCEAN, color='black')
# GRIDS
# ax.set_extent([LAT_BOUNDS[1]-1, LAT_BOUNDS[1]+1, LAT_BOUNDS[0]-1, LAT_BOUNDS[0]+1])
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--', x_inline=False, y_inline=False)
gl.xlocator = mticker.FixedLocator([20, 60, 100, 140])
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

scat = ax.scatter(np.array(GRID_TO_COORDS)[:,1],np.array(GRID_TO_COORDS)[:,0],c=weights,vmax = 20, s=1, cmap="inferno")

cbar = plt.colorbar(scat,fraction=0.0415, pad=0.04)
cbar.set_label(f'Counts of resulting associations{" (log)" if log else ""}', rotation=270, labelpad=20)
ax.set_title(f"Resulting map")
ax.set_xlabel("lon (°)")
ax.set_ylabel("lat (°)")


## Localisation

In [ ]:

# del DETECTIONS_MERGED , DETECTIONS
files = glob2.glob(f"{ASSOCIATION_OUTPUT_DIR}/{DETECTIONS_DIR_NAME}/cache/associations.pkl")
with open(files[0], "rb") as f:
    association = pickle.load(f)

def process(i):
    station = list(map(lambda j: STATIONS[j].get_pos(), association[i][0][:,0]))
    if len(station) < 3:
        return None
    time_elapsed_seconds = 365.25*24*3600*1
    drift = [e.other_kwargs["clock_drift_ppm"] * 1e-6 * time_elapsed_seconds  if e.other_kwargs['gps_sync'] != 'ok' else 0 for e in event_extracted.station]
    drift = np.abs(drift)
    det = list(map(lambda j: IDX_TO_DET[j][0], association[i][0][:,1]))
    min_date = np.argmin(det)
    c0 = list(map(lambda j: GRID_TO_COORDS[j], association[i][1]))
    t0 = -1 * SOUND_MODEL.get_sound_travel_time(np.mean(c0, axis =0), station[min_date], det[min_date])
    x0 = [t0]+list(np.mean(c0, axis =0))
    pick_uncertainties = [3]*len(station)
    res = SOUND_MODEL.localize_with_uncertainties(
        station, det,y_min=lon_min-6, x_min=lat_min-6,y_max=lon_max+6,x_max=lat_max+6, initial_pos=x0,pick_uncertainties = pick_uncertainties, drift_uncertainties = drift
    )
    return i, res

# Taille du chunk (param important à ajuster selon ton CPU/RAM)
CHUNK_SIZE = 25
results = {}
with mp.Pool(mp.cpu_count()) as pool, open("results.npy", "wb") as f:
    for r in tqdm(pool.imap(process, range(len(association)), chunksize=CHUNK_SIZE),
                  total=len(association)):
        if r is not None:
            i, res = r
            results[i] = res

with open("results_simu.pkl", "wb") as f:
    pickle.dump(results, f)

In [ ]:
from datetime import tzinfo

datetime.fromtimestamp(results[0].x[0]).date()

In [ ]:
from collections import Counter
from itertools import combinations


# Ajouter les fonctions d'évaluation de qualité
def calculate_reduced_chi_square(res):
    """Calcule le chi-carré réduit à partir du résultat de l'optimisation."""
    n_observations = len(res.fun)
    n_parameters = 2
    degrees_of_freedom = n_observations - n_parameters
    chi_square = np.sum(res.fun**2)
    reduced_chi_square = chi_square / degrees_of_freedom if degrees_of_freedom > 0 else float('inf')
    return reduced_chi_square, chi_square, degrees_of_freedom

def calculate_azimuthal_gap(sensors_positions, event_position):
    """Calcule le gap azimutal maximal entre les stations."""
    azimuths = []
    for sensor_pos in sensors_positions:

        dx = sensor_pos.get_pos()[1] - event_position[1]
        dy = sensor_pos.get_pos()[0] - event_position[0]
        azimuth = np.degrees(np.arctan2(dx, dy)) % 360
        azimuths.append(azimuth)

    azimuths_sorted = sorted(azimuths)
    gaps = []
    for i in range(len(azimuths_sorted)):
        next_idx = (i + 1) % len(azimuths_sorted)
        gap = (azimuths_sorted[next_idx] - azimuths_sorted[i]) % 360
        gaps.append(gap)

    return max(gaps) if gaps else 360

def estimate_position_uncertainty(res):
    """Estime l'incertitude sur la position à partir de la matrice de covariance."""
    try:
        J = res.jac
        covariance_matrix = np.linalg.inv(J.T @ J) * (2 * res.cost) / (len(res.fun) - 2)
        x_uncertainty = np.sqrt(covariance_matrix[0, 0])
        y_uncertainty = np.sqrt(covariance_matrix[1, 1])
        return (x_uncertainty, y_uncertainty), covariance_matrix
    except:
        return (float('inf'), float('inf')), None

def quality_score(res, sensors_positions, weights=None, max_chi_sq=3.0, max_gap=180.0):
    """Calcule un score de qualité combinant plusieurs indicateurs."""
    # Reduced chi-square
    red_chi_sq, _, _ = calculate_reduced_chi_square(res)
    chi_sq_score = max(0, 1 - (red_chi_sq / max_chi_sq))

    # Gap azimutal
    event_pos = (res.x[1], res.x[2]) if len(res.x) > 2 else (res.x[0], res.x[1])
    gap = calculate_azimuthal_gap(sensors_positions, event_pos)
    gap_score = max(0, 1 - (gap / max_gap))

    # Résidus standardisés
    std_residuals = res.fun
    outlier_score = 1 - (np.sum(np.abs(std_residuals) > 3.0) / len(std_residuals))

    # Score combiné (pondéré)
    combined_score = 0.5 * chi_sq_score + 0.3 * gap_score + 0.2 * outlier_score
    return combined_score, {
        'red_chi_sq': red_chi_sq,
        'azimuthal_gap': gap,
        'chi_sq_score': chi_sq_score,
        'gap_score': gap_score,
        'outlier_score': outlier_score
    }



def filter_by_cost_threshold(results_dict, cost_threshold=10000):
    """
    Filtre les associations par seuil de coût.

    Args:
        results_dict: Dictionnaire des résultats d'optimisation
        cost_threshold: Seuil de coût maximum

    Returns:
        dict: results_dict filtré par coût
    """
    print(f"Step 1: Filtering by cost threshold ({cost_threshold})...")

    filtered_results = {}
    for i, res in results_dict.items():
        cost = res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)
        if cost < cost_threshold:
            filtered_results[i] = res

    print(f"Results after cost filtering: {len(filtered_results)}")
    return filtered_results


def remove_subset_associations(results_dict, association, keep_best_duplicate=True):
    """
    Supprime les associations qui utilisent un sous-ensemble de détections
    par rapport à d'autres associations.

    Args:
        results_dict: Dictionnaire des résultats d'optimisation
        association: Structure contenant les associations de détections
        keep_best_duplicate: Si True, garde seulement la meilleure association
                            pour les ensembles de détections identiques

    Returns:
        tuple: (results_dict filtré, set des indices exclus)
    """
    print("Step 2: Removing subset associations...")

    # Préparer les données avec les sets de détections
    events_data = []
    for i in results_dict.keys():
        detections = association[i][0][:, 1]  # Index des détections
        det_set = frozenset(detections)
        cost = results_dict[i].cost if hasattr(results_dict[i], 'cost') else np.sum(results_dict[i].fun**2)
        date = datetime.fromtimestamp(results_dict[i].x[0]).date()
        events_data.append({
            'uid': i,
            "date": date,
            'detection_set': det_set,
            'cost': cost,
            'set_size': len(det_set)
        })

    # Trier par taille décroissante, puis par coût croissant
    print("Sorting by detection set size (descending) and cost (ascending)...")
    events_data.sort(key=lambda x: (x['date'], -x['set_size'], x['cost']))

    # Algorithme de suppression des subsets
    accepted_events = defaultdict(list)
    excluded_indices = set()
    seen_detection_sets = set()

    for current in tqdm(events_data, desc="Filtering subsets"):
        current_set = current['detection_set']
        date = current['date']
        is_subset = False

        # Vérifier si on a déjà vu exactement le même ensemble
        if keep_best_duplicate and current_set in seen_detection_sets:
            excluded_indices.add(current['uid'])
            is_subset = True
        else:
            # Vérifier si current est un sous-ensemble d'une association acceptée
            for accepted in accepted_events[date-timedelta(days=-1)] + accepted_events[date]+ accepted_events[date+timedelta(days=1)]:
                if current_set.issubset(accepted['detection_set']):
                    # Si c'est un sous-ensemble strict OU un doublon qu'on veut éliminer
                    if current_set != accepted['detection_set'] or keep_best_duplicate:
                        excluded_indices.add(current['uid'])
                        is_subset = True
                        break

        if not is_subset:
            accepted_events[date].append(current)
            seen_detection_sets.add(current_set)

    print(f"Events after subset removal: {len(accepted_events)}")
    print(f"Excluded subset events: {len(excluded_indices)}")
    accepted_events = [x for xs in accepted_events.values() for x in xs]
    # Créer le dictionnaire filtré
    filtered_results = {event['uid']: results_dict[event['uid']] for event in accepted_events}

    return filtered_results, excluded_indices


# Version alternative avec optimisation spatiale pour grands datasets
def remove_subset_associations_optimized(results_dict, association, keep_best_duplicate=True):
    """
    Version optimisée pour les grands datasets utilisant un arbre de prefix (trie)
    """
    print("Step 2: Removing subset associations (optimized version)...")

    # Préparer les données
    events_data = []
    for i in results_dict.keys():
        try:
            arr = np.asarray(association[i][0])
            detections = arr[:, 1] if arr.ndim == 2 and arr.size > 0 else np.array([], dtype=int)
        except (IndexError, ValueError):
            detections = np.array([], dtype=int)

        det_set = frozenset(detections)
        cost = results_dict[i].cost if hasattr(results_dict[i], 'cost') else np.sum(results_dict[i].fun**2)

        events_data.append({
            'uid': i,
            'detection_set': det_set,
            'set_size': len(det_set),
            'cost': cost
        })

    # Trier par taille décroissante et coût croissant
    events_data.sort(key=lambda x: (-x['set_size'], x['cost']))

    # Utiliser un arbre de préfixe pour les ensembles
    class DetectionSetTrie:
        def __init__(self):
            self.children = {}
            self.is_end = False
            self.event = None

        def insert(self, detection_set, event):
            """Insère un ensemble trié"""
            sorted_dets = sorted(detection_set)
            node = self
            for det in sorted_dets:
                if det not in node.children:
                    node.children[det] = DetectionSetTrie()
                node = node.children[det]
            node.is_end = True
            node.event = event

        def is_subset_of_existing(self, detection_set):
            """Vérifie si l'ensemble est sous-ensemble d'un ensemble existant"""
            sorted_dets = sorted(detection_set)
            return self._is_subset(sorted_dets, 0)

        def _is_subset(self, sorted_dets, index):
            if index >= len(sorted_dets):
                return True

            current_det = sorted_dets[index]
            for det, child in self.children.items():
                if det <= current_det:
                    if det == current_det:
                        if child._is_subset(sorted_dets, index + 1):
                            return True
                    else:
                        if child._is_subset(sorted_dets, index):
                            return True
            return False

    trie = DetectionSetTrie()
    accepted_events = []
    excluded_indices = set()

    print("Processing events with prefix tree...")
    for event in tqdm(events_data, desc="Checking subsets"):
        current_set = event['detection_set']

        if not current_set:  # Ensemble vide
            if keep_best_duplicate:
                excluded_indices.add(event['uid'])
                continue
            else:
                accepted_events.append(event)
                trie.insert(current_set, event)
            continue

        # Vérifier si c'est un sous-ensemble
        if trie.is_subset_of_existing(current_set):
            excluded_indices.add(event['uid'])
        else:
            # Vérifier les doublons exacts
            is_duplicate = False
            if keep_best_duplicate:
                for accepted in accepted_events:
                    if accepted['detection_set'] == current_set:
                        # Garder celui avec le meilleur coût
                        if event['cost'] < accepted['cost']:
                            accepted_events.remove(accepted)
                            excluded_indices.add(accepted['uid'])
                            accepted_events.append(event)
                            trie.insert(current_set, event)  # Mettre à jour le trie
                        else:
                            excluded_indices.add(event['uid'])
                        is_duplicate = True
                        break

            if not is_duplicate:
                accepted_events.append(event)
                trie.insert(current_set, event)

    print(f"Events after subset removal: {len(accepted_events)}")
    print(f"Excluded subset events: {len(excluded_indices)}")

    filtered_results = {event['uid']: results_dict[event['uid']] for event in accepted_events}
    return filtered_results, excluded_indices

#
# def remove_overlap(results_dict, association):
#     print("Step 3: Handling overlapping associations with residual analysis...")
#
#     def analyze_residuals(result):
#         """Analyse la qualité des résidus pour évaluer la pertinence de l'association"""
#         residuals = result.fun
#         num_obs = len(residuals)
#         chi2_stat, p_value, passes, dof = SOUND_MODEL.test_chi_square(result)
#         if num_obs == 0:
#             print("error")
#         # Chi-carré réduit
#         chi2_reduced = np.sum(residuals**2) / dof
#         # Z-scroe des résidus (résidus standardisés)
#         residual_std = np.std(residuals) if np.std(residuals) > 1e-10 else 1e-10
#         z_scores = np.abs(residuals) / residual_std
#         max_zscore = np.max(z_scores)
#         outliers = SOUND_MODEL.detect_outliers(result,method='absolute')
#         outliers_score = len(outliers['outlier_indices'])
#         # Fraction d'outliers (|z-score| > 2.5)
#         outlier_fraction = np.mean(z_scores >1.5)
#
#         # Test unitaire de variance
#         p = 2  # nombre de paramètres ajustés (ici a et b)
#         sigma_theorique = 3
#         s2 = np.var(residuals, ddof=p)  # variance sans biais
#         # print(f"Variance empirique : {s2:.3f}")
#         # Test de normalité des résidus (Shapiro-Wilk si n < 5000, sinon Kolmogorov-Smirnov)
#         if num_obs >= 3 and num_obs < 5000:
#              _, normality_pvalue = stats.shapiro(residuals)
#         # Score de qualité composite (plus c'est haut, mieux c'est)
#         quality_score = 1.0 / (1.0 + chi2_reduced)  # Pénalise chi2 élevé
#         quality_score *= (1.0 - outlier_fraction)    # Pénalise les outliers
#         quality_score *= 1.0 / (1.0 + max_zscore/10.0)  # Pénalise les résidus extrêmes
#         return {
#             'chi2_reduced': float(chi2_reduced),
#             'max_residual_zscore': float(max_zscore),
#             'outlier_fraction': float(outlier_fraction),
#             'normality_pvalue': float(normality_pvalue),
#             'quality_score': float(quality_score),
#             'cost':float(result.cost),
#             'chi2_stat': float(chi2_stat),
#             'p_value': float(p_value),
#             'passes': bool(passes),
#             "outliers_score" : outliers_score
#         }
#     # Analyse des événements avec résidus
#     events = {}
#     print("Analyzing residuals for each association...")
#
#     for i in tqdm(results_dict.keys(), desc="Processing associations"):
#         try:
#             arr = np.asarray(association[i][0])
#             if arr.size == 0:
#                 detections = np.array([], dtype=int)
#             elif arr.ndim == 1:
#                 detections = np.array([int(arr[1])]) if arr.size >= 2 else np.array([], dtype=int)
#             else:
#                 detections = arr[:, 1].astype(int)
#         except Exception:
#             detections = np.array([], dtype=int)
#             arr = np.array([])
#
#         det_set = frozenset(detections.tolist())
#         num_stations = int(arr.shape[0]) if hasattr(arr, 'shape') else 0
#         node = results_dict[i]
#
#         # Calcul du coût traditionnel
#         cost = getattr(node, 'cost', None)
#         if cost is None:
#             fun = getattr(node, 'fun', None)
#             cost = float(np.sum(fun**2)) if fun is not None else float('inf')
#
#         # Analyse des résidus
#         residual_analysis = analyze_residuals(node)
#
#         events[i] = {
#             'detection_set': det_set,
#             'cost': float(cost),
#             'num_stations': int(num_stations),
#             'residual_analysis': residual_analysis
#         }
#
#     # Filtrage préliminaire basé sur la qualité des résidus
#     quality_threshold = np.percentile([ev['residual_analysis']['quality_score']
#                                      for ev in events.values()], 10)  # Garde les 90% meilleurs
#
#     print(f"Quality threshold: {quality_threshold:.4f}")
#
#
#     det_to_events = defaultdict(set)
#     for uid, ev in events.items():
#         for d in ev['detection_set']:
#             det_to_events[d].add(uid)
#
#     G = nx.Graph()
#     G.add_nodes_from(events.keys())
#     for uids in det_to_events.values():
#         if len(uids) > 1:
#             for a, b in combinations(sorted(uids, key=lambda x: str(x)), 2):
#                 G.add_edge(a, b)
#
#     print(f"Found {G.number_of_edges()} overlapping association pairs")
#
#     clusters = list(nx.connected_components(G))
#     overlapping_clusters = [c for c in clusters if len(c) > 1]
#     isolated_events = [list(c)[0] for c in clusters if len(c) == 1]
#
#     def enhanced_association_score(uid):
#         """Score amélioré intégrant l'analyse des résidus"""
#         ev = events[uid]
#
#         # Score traditionnel
#         traditional_score = ev['cost'] / (ev['num_stations'] + 1e-9)
#
#         # Poids basé sur la qualité des résidus
#         quality_weight = ev['residual_analysis']['quality_score']
#
#         # Score composite (plus petit = meilleur)
#         composite_score = -quality_weight
#         outliers_score = ev['residual_analysis']['outliers_score']
#         return (outliers_score,
#             composite_score*quality_weight, # Chi2 réduit
#             # ev['residual_analysis']['outlier_fraction'],
#             # ev['residual_analysis']['chi2_reduced'],
#             # traditional_score,# Qualité (négatif pour ordre croissant
#         )
#
#     final_indices = []
#
#     # Traitement des clusters avec overlap
#     for cluster in tqdm(overlapping_clusters, desc="Processing overlapping clusters"):
#         cluster = set(cluster)
#         events_dets = {uid: events[uid]['detection_set'] for uid in cluster}
#
#         # Sous-clustering basé sur la similarité
#         H = nx.Graph()
#         H.add_nodes_from(cluster)
#         for a, b in combinations(sorted(cluster, key=lambda x: str(x)), 2):
#             A = events_dets[a]
#             B = events_dets[b]
#             inter = len(A & B)
#             union = len(A | B)
#             jaccard = inter / union if union else 0.0
#
#             # Critères pour lier deux événements
#             if inter >= 2 or jaccard >= 0.1:
#                 H.add_edge(a, b)
#
#         subs = list(nx.connected_components(H))
#
#         for sub in subs:
#             sub = set(sub)
#             ordered = sorted(list(sub), key=enhanced_association_score)
#
#             # Sélection gloutonne dans le sous-cluster
#             picked = []
#             picked_dets = set()
#
#             for uid in ordered:
#                 dets = events[uid]['detection_set']
#                 quality_score = events[uid]['residual_analysis']['quality_score']
#                 passes = events[uid]['residual_analysis']['passes']
#                 # Vérification des critères de sélection
#                 has_overlap = bool(dets & picked_dets)
#                 is_high_quality = passes#quality_score >= quality_threshold * 0.5  # Seuil plus souple
#
#                 if not has_overlap and is_high_quality:
#                     picked.append(uid)
#                     picked_dets |= set(dets)
#
#             if not picked:
#                 # Si aucun n'est sélectionné, prend le meilleur du sous-cluster
#                 best = min(sub, key=enhanced_association_score)
#                 final_indices.append(best)
#             else:
#                 final_indices.extend(picked)
#
#     final_indices.extend(isolated_events)
#     final_indices = list(dict.fromkeys(final_indices))
#
#     # Sélection finale avec analyse des résidus
#     candidates = list(dict.fromkeys(final_indices))
#     candidates.sort(key=enhanced_association_score)
#
#     selected = []
#     selected_dets = set()
#
#     print("Final selection with residual quality check...")
#     for uid in tqdm(candidates, desc="Final selection"):
#         dets = events[uid]['detection_set']
#         quality_score = events[uid]['residual_analysis']['quality_score']
#
#         # Critères de sélection finale
#         no_overlap = selected_dets.isdisjoint(dets)
#         good_quality = quality_score >= quality_threshold * 0.3  # Seuil très souple pour la sélection finale
#         reasonable_residuals = events[uid]['residual_analysis']['max_residual_zscore'] < 5.0
#
#         if no_overlap and good_quality and reasonable_residuals:
#             selected.append(uid)
#             selected_dets |= set(dets)
#
#     # Validation finale
#     total_dets = sum(len(events[uid]['detection_set']) for uid in selected)
#     if total_dets != len(selected_dets):
#         raise RuntimeError("Overlap detected after final selection")
#
#     # Statistiques sur la qualité
#     if selected:
#         quality_scores = [events[uid]['residual_analysis']['quality_score'] for uid in selected]
#         chi2_values = [events[uid]['residual_analysis']['chi2_reduced'] for uid in selected]
#
#         print(f"Selected {len(selected)} associations")
#         print(f"Quality scores: mean={np.mean(quality_scores):.4f}, std={np.std(quality_scores):.4f}")
#         print(f"Chi2 reduced: mean={np.mean(chi2_values):.4f}, std={np.std(chi2_values):.4f}")
#     else:
#         print("No associations selected!")
#
#     filtered_results = {i: results_dict[i] for i in selected if i in results_dict}
#     return filtered_results

def remove_overlap(results_dict, association):
    print("Step 3: Advanced filtering of localization results...")

    def compute_quality_metrics(result):
        """Calcule des métriques de qualité robustes pour évaluer la localisation"""
        residuals = result.fun
        num_obs = len(residuals)

        if num_obs < 3:  # Trop peu d'observations
            return {
                'quality_score': -np.inf,
                'chi2_reduced': np.inf,
                'outlier_fraction': 1.0,
                'max_residual_zscore': np.inf,
                'passes_chi2': False,
                'num_obs': num_obs,
                'cost': np.inf
            }

        # Test chi-carré
        chi2_stat, p_value, passes_chi2, dof = SOUND_MODEL.test_chi_square(result)

        # Métriques principales de qualité
        chi2_reduced = np.sum(residuals**2) / dof if dof > 0 else np.inf

        # Détection robuste des outliers avec MAD (Median Absolute Deviation)
        median_residual = np.median(residuals)
        mad = np.median(np.abs(residuals - median_residual))
        robust_std = mad * 1.4826 if mad > 0 else 1e-10  # Conversion MAD → std normal

        # Z-scores robustes
        z_scores = np.abs(residuals - median_residual) / robust_std
        max_zscore = np.max(z_scores)

        # Fraction d'outliers avec seuil robuste
        outlier_fraction = np.mean(z_scores > 2.5)  # Seuil plus conservateur

        # Score de qualité composite (pondéré)
        weights = {
            'p_value': 0.35,
            'chi2': 0.25,
            'outliers': 0.25,
            'max_residual': 0.1,
            'num_obs': 0.15
        }

        # Composantes normalisées du score
        chi2_component = 1.0 / (1.0 + min(chi2_reduced, 10.0))  # Pénalise chi2 élevé
        outlier_component = 1.0 - outlier_fraction
        residual_component = 1.0 / (1.0 + max_zscore / 6.0)  # Pénalise résidus extrêmes
        obs_component = min(num_obs / 8.0, 1.0)  # Favorise plus d'observations
        p_value_component = min(p_value*2, 1.0) # Normaliser p∈[0,1]

        quality_score = (
            weights['p_value']*p_value_component+
            weights['chi2'] * chi2_component +
            weights['outliers'] * outlier_component +
            # weights['max_residual'] * residual_component +
            weights['num_obs'] * obs_component
        )

        return {
            'quality_score': float(quality_score),
            'chi2_reduced': float(chi2_reduced),
            'max_residual_zscore': float(max_zscore),
            'outlier_fraction': float(outlier_fraction),
            'passes_chi2': bool(passes_chi2),
            'num_obs': int(num_obs),
            'cost': float(np.sum(residuals**2)),
            'p_value': float(p_value)
        }

    # Construction du graphe d'événements
    print("Building event graph and computing quality metrics...")
    events = {}
    det_to_events = defaultdict(set)

    for i in tqdm(results_dict.keys(), desc="Processing associations"):
        try:
            arr = np.asarray(association[i][0])
            detections = arr[:, 1].astype(int) if arr.ndim == 2 and arr.size > 0 else np.array([], dtype=int)
            num_stations = arr.shape[0] if arr.ndim == 2 else 0
        except Exception:
            detections = np.array([], dtype=int)
            num_stations = 0

        det_set = frozenset(detections.tolist())
        quality_metrics = compute_quality_metrics(results_dict[i])

        events[i] = {
            'detection_set': det_set,
            'num_stations': num_stations,
            'quality': quality_metrics
        }

        # Mapping détection → événements
        for det in det_set:
            det_to_events[det].add(i)

    # Construction du graphe de chevauchement
    G = nx.Graph()
    G.add_nodes_from(events.keys())

    for det, event_set in det_to_events.items():
        if len(event_set) > 1:
            for event1, event2 in combinations(sorted(event_set), 2):
                G.add_edge(event1, event2)

    print(f"Overlap graph: {G.number_of_nodes()} events, {G.number_of_edges()} overlaps")

    # Séparation des événements isolés et groupés
    clusters = list(nx.connected_components(G))
    isolated_events = [list(c)[0] for c in clusters if len(c) == 1]
    overlapping_clusters = [c for c in clusters if len(c) > 1]

    print(f"Found {len(isolated_events)} isolated events and {len(overlapping_clusters)} overlapping clusters")

    def event_priority_score(event_id):
        """Score de priorité pour la sélection (plus élevé = meilleur)"""
        ev = events[event_id]
        qual = ev['quality']

        # Critères de rejet immédiat
        if qual['num_obs'] < 3:  # Trop peu de données
            return (-np.inf, 0, 0, 0)
        if not qual['passes_chi2']:  # Échec test chi2
            return (-np.inf, 0, 0, 0)
        if qual['max_residual_zscore'] > 8.0:  # Résidu trop extrême
            return (-np.inf, 0, 0, 0)

        # Score principal basé sur la qualité
        base_score = qual['quality_score']

        # Bonus pour le nombre de stations (diversité des données)
        station_bonus = min(ev['num_stations'] / 10.0, 1.0)

        # Pénalité pour le chi2 élevé
        chi2_penalty = 1.0 / (1.0 + qual['chi2_reduced'])

        # Score composite
        composite_score = (
            base_score * 0.6 +
            station_bonus * 0.2 +
            chi2_penalty * 0.2
        )

        return (
            composite_score,  # Score principal
            qual['p_value']
            -qual['chi2_reduced'],  # Chi2 (plus bas = mieux)
            -qual['outlier_fraction'],  # Fraction d'outliers (plus bas = mieux)
            ev['num_stations']  # Nombre de stations
        )

    selected_events = set()
    selected_detections = set()

    # Traitement des clusters avec chevauchement
    print("Resolving overlapping clusters...")
    for cluster in tqdm(overlapping_clusters, desc="Processing clusters"):
        cluster_events = list(cluster)

        # Tri par qualité décroissante
        cluster_events.sort(key=event_priority_score, reverse=True)

        # Sélection gloutonne optimisée
        cluster_selected = []
        cluster_used_dets = set()

        for event_id in cluster_events:
            event_dets = events[event_id]['detection_set']

            # Vérifier le chevauchement et la qualité
            has_overlap = not event_dets.isdisjoint(cluster_used_dets)
            is_high_quality = event_priority_score(event_id)[0] > -np.inf

            if not has_overlap and is_high_quality:
                cluster_selected.append(event_id)
                cluster_used_dets.update(event_dets)

        # Si conflit persistant, prendre le meilleur événement du cluster
        if not cluster_selected and cluster_events:
            best_event = cluster_events[0]
            if event_priority_score(best_event)[0] > -np.inf:
                cluster_selected = [best_event]
                cluster_used_dets = events[best_event]['detection_set']

        selected_events.update(cluster_selected)
        selected_detections.update(cluster_used_dets)

    # Ajout des événements isolés de bonne qualité
    print("Adding qualified isolated events...")
    for event_id in isolated_events:
        if (event_priority_score(event_id)[0] > -np.inf and
            events[event_id]['detection_set'].isdisjoint(selected_detections)):
            selected_events.add(event_id)
            selected_detections.update(events[event_id]['detection_set'])

    # Validation finale et statistiques
    final_events = list(selected_events)

    # Vérification de cohérence
    total_dets_in_events = set()
    for event_id in final_events:
        total_dets_in_events.update(events[event_id]['detection_set'])

    if total_dets_in_events != selected_detections:
        print("Warning: Inconsistency in detection tracking")
        # Correction automatique
        selected_detections = total_dets_in_events

    # Statistiques de qualité finale
    if final_events:
        quality_scores = [events[eid]['quality']['quality_score'] for eid in final_events]
        chi2_values = [events[eid]['quality']['chi2_reduced'] for eid in final_events]
        num_stations_list = [events[eid]['num_stations'] for eid in final_events]

        print(f"\n=== FILTERING RESULTS ===")
        print(f"Selected {len(final_events)} events from {len(events)} candidates")
        print(f"Quality scores: mean={np.mean(quality_scores):.3f} ± {np.std(quality_scores):.3f}")
        print(f"Chi2 reduced: mean={np.mean(chi2_values):.3f} ± {np.std(chi2_values):.3f}")
        print(f"Stations per event: mean={np.mean(num_stations_list):.1f} ± {np.std(num_stations_list):.1f}")
        print(f"Detection reuse efficiency: {len(selected_detections)} detections for {len(final_events)} events")

        # Analyse des rejets
        rejected = set(events.keys()) - set(final_events)
        if rejected:
            reject_reasons = []
            for rid in rejected:
                qual = events[rid]['quality']
                if qual['num_obs'] < 3:
                    reject_reasons.append("too_few_observations")
                elif not qual['passes_chi2']:
                    reject_reasons.append("failed_chi2_test")
                elif qual['max_residual_zscore'] > 8.0:
                    reject_reasons.append("extreme_residuals")
                else:
                    reject_reasons.append("overlap_or_quality")

            print(f"Rejection reasons: {dict(Counter(reject_reasons))}")
    else:
        print("Warning: No events selected after filtering!")
        # Fallback: prendre les meilleurs événements non chevauchants
        final_events = fallback_selection(events, selected_detections)

    filtered_results = {eid: results_dict[eid] for eid in final_events if eid in results_dict}
    return filtered_results

def fallback_selection(events, used_detections):
    """Sélection de repli si la sélection principale échoue"""
    candidates = []
    for event_id, ev_data in events.items():
        if (ev_data['quality']['num_obs'] >= 3 and
            ev_data['detection_set'].isdisjoint(used_detections)):
            candidates.append(event_id)

    # Tri par qualité
    candidates.sort(key=lambda x: events[x]['quality']['quality_score'], reverse=True)

    selected = []
    current_dets = set(used_detections)

    for cand in candidates:
        cand_dets = events[cand]['detection_set']
        if current_dets.isdisjoint(cand_dets):
            selected.append(cand)
            current_dets.update(cand_dets)

    return selected

def filter_associations_simple(results_dict, association, cost_threshold=10000):
    """
    Version améliorée du filtrage des associations avec fonctions séparées.

    Args:
        results_dict: Dictionnaire des résultats d'optimisation
        association: Structure contenant les associations de détections
        cost_threshold: Seuil de coût pour le filtrage initial

    Returns:
        dict: Dictionnaire des résultats filtrés
    """
    print("Starting association filtering...")
    print(f"Initial associations count: {len(results_dict)}")

    # Étape 1: Filtrer par seuil de coût
    filtered_by_cost = filter_by_cost_threshold(results_dict, cost_threshold)
    # filtered_by_cost = results_dict
    # Étape 2: Supprimer les associations subset
    filtered_no_subsets, excluded_indices = remove_subset_associations(filtered_by_cost, association)
    # filtered_no_subsets =results_dict
    # excluded_indices = []
    # Étape 3: Résoudre les overlaps
    final_results = remove_overlap(filtered_no_subsets, association)
    # final_results = filtered_no_subsets
    print("Association filtering completed!")
    print(f"Final results: {len(final_results)} associations")

    return final_results, excluded_indices

def run_filtered_associations():
    """Pipeline complet avec filtrage - version simplifiée"""
    CHUNK_SIZE = 25

    # # Étape 1: Traiter toutes les associations avec dictionnaire
    # print("Step 1: Processing all associations...")
    # results = {}
    # with mp.Pool(mp.cpu_count()) as pool:
    #     for r in tqdm(pool.imap(process, range(len(association)), chunksize=CHUNK_SIZE),
    #                   total=len(association)):
    #         if r is not None:
    #             i, res = r
    #             results[i] = res

    print(f"Raw associations processed: {len(results)}")
    with open("results_sium.pkl", "wb") as f:
        pickle.dump(results, f)
    # Étape 2: Filtrer et déduplicater
    filtered_results, excluded_indices = filter_associations_simple(results,association, cost_threshold= 900)

    # Étape 3: Sauvegarder
    # Sauvegarder juste les résultats comme array (compatible avec votre format original)
    results_array = [filtered_results[i] for i in sorted(filtered_results.keys())]
    np.save("filtered_results.npy", np.array(results_array))

    # Optionnel: sauvegarder les métadonnées
    metadata = {
        'filtered_indices': list(filtered_results.keys()),
        'excluded_indices': list(excluded_indices),
        'original_count': len(association),
        'processed_count': len(results),
        'final_count': len(filtered_results)
    }
    np.save("association_metadata.npy", metadata)

    print(f"Processing complete:")
    print(f"  Original associations: {len(association)}")
    print(f"  Successfully processed: {len(results)}")
    print(f"  After filtering: {len(filtered_results)}")
    print(f"  Excluded: {len(excluded_indices)}")

    return filtered_results

# Utilisation
if __name__ == "__main__":
    final_results = run_filtered_associations()

In [ ]:
def plot_association_map(results, stations_dict, title="Associations localisées"):
    """
    Carte montrant les positions des événements et des stations
    """
    fig, ax = plt.subplots(figsize=(12, 8))

    # Extraire les positions des événements
    event_positions = []
    costs = []
    is_valid = []

    for i, res in results.items():
        if hasattr(res, 'x') and len(res.x) >= 2:
            # Supposer que res.x = [t0, lat, lon] ou [lat, lon]
            if len(res.x) == 3:
                lat, lon = res.x[1], res.x[2]
            else:
                lat, lon = res.x[0], res.x[1]
            event_positions.append([lat, lon])
            cost = res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)
            costs.append(cost)

    if event_positions:
        event_positions = np.array(event_positions)

        costs = np.array(costs)

        # Scatter plot des événements colorés par cost
        scatter = ax.scatter(event_positions[:, 1], event_positions[:, 0],
                           c=costs, cmap='viridis_r', s=50, alpha=0.7,
                           label='Événements')
        plt.colorbar(scatter, label='Cost')

        # Ajouter les stations si disponibles
        if stations_dict:
            station_lats = []
            station_lons = []
            for station in stations_dict:
                pos = station.get_pos()
                station_lats.append(pos[0])
                station_lons.append(pos[1])

            ax.scatter(station_lons, station_lats, c='red', marker='^',
                      s=100, label='Stations', alpha=0.8)

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

def plot_cost_distribution(results, bins=50):
    """
    Distribution des costs
    """
    costs = []
    red_chi_squares = []

    for i, res in results.items():
        cost = res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)
        costs.append(cost)

        # Calculer reduced chi-square
        n_obs = len(res.fun)
        n_params = 2
        dof = n_obs - n_params
        red_chi_sq = cost / dof if dof > 0 else float('inf')
        red_chi_squares.append(red_chi_sq)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Distribution des costs
    ax1.hist(costs, bins=bins, alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Cost')
    ax1.set_ylabel('Nombre d\'associations')
    ax1.set_title('Distribution des Costs')
    ax1.axvline(np.median(costs), color='red', linestyle='--',
                label=f'Médiane: {np.median(costs):.1f}')
    ax1.legend()

    # Distribution des reduced chi-square
    red_chi_squares = [x for x in red_chi_squares if x < 100]  # Filtrer les valeurs extrêmes
    ax2.hist(red_chi_squares, bins=bins, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Reduced Chi-square')
    ax2.set_ylabel('Nombre d\'associations')
    ax2.set_title('Distribution des Reduced Chi-square')
    ax2.axvline(1.0, color='green', linestyle='--', label='Valeur idéale')
    ax2.axvline(np.median(red_chi_squares), color='red', linestyle='--',
                label=f'Médiane: {np.median(red_chi_squares):.2f}')
    ax2.legend()

    plt.tight_layout()
    return fig, costs, red_chi_squares

def plot_residuals_analysis(results, association_data, idx_to_det, stations_dict):
    """
    Analyse des résidus par station
    """
    station_residuals = {}

    for i, res in results.items():
        if i in association_data:
            stations_idx = association_data[i][0][:,0]
            residuals = res.fun

            for j, station_idx in enumerate(stations_idx):
                if j < len(residuals):
                    if station_idx not in station_residuals:
                        station_residuals[station_idx] = []
                    station_residuals[station_idx].append(residuals[j])

    # Box plot des résidus par station
    fig, ax = plt.subplots(figsize=(12, 6))

    station_names = []
    residual_data = []

    for station_idx, residuals in station_residuals.items():
        if len(residuals) >= 5:  # Au moins 5 mesures
            station_names.append(f'Station {station_idx}')
            residual_data.append(residuals)

    if residual_data:
        ax.boxplot(residual_data, labels=station_names)
        ax.set_ylabel('Résidus (s)')
        ax.set_title('Distribution des résidus par station')
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)

    plt.tight_layout()
    return fig

def plot_quality_metrics(results, association_data, stations_dict):
    """
    Métriques de qualité des associations
    """
    quality_data = []

    for i, res in results.items():
        try:
            if True :
                # Récupérer les positions des stations
                stations_idx = association_data[i][0][:,0]
                stations_pos = [stations_dict[idx].get_pos() for idx in stations_idx]

                # Calculer les métriques de base
                cost = res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)

                # Reduced chi-square
                n_obs = len(res.fun)
                n_params = 2
                dof = n_obs - n_params
                red_chi_sq = cost / dof if dof > 0 else float('inf')

                # Gap azimutal
                event_pos = (res.x[1], res.x[2]) if len(res.x) > 2 else (res.x[0], res.x[1])

                # Calculer azimuts
                azimuths = []
                for sensor_pos in stations_pos:
                    dx = sensor_pos[1] - event_pos[1]
                    dy = sensor_pos[0] - event_pos[0]
                    azimuth = np.degrees(np.arctan2(dx, dy)) % 360
                    azimuths.append(azimuth)

                # Calculer gap azimutal
                azimuths_sorted = sorted(azimuths)
                gaps = []
                for j in range(len(azimuths_sorted)):
                    next_idx = (j + 1) % len(azimuths_sorted)
                    gap = (azimuths_sorted[next_idx] - azimuths_sorted[j]) % 360
                    gaps.append(gap)
                az_gap = max(gaps) if gaps else 360

                # Incertitude position (approximation simple)
                try:
                    if hasattr(res, 'jac') and res.jac is not None:
                        J = res.jac
                        cov_matrix = np.linalg.inv(J.T @ J) * (2 * cost) / max(dof, 1)
                        max_uncert = np.sqrt(max(np.diag(cov_matrix)))
                    else:
                        max_uncert = None
                except:
                    max_uncert = None

                quality_data.append({
                    'association_id': i,
                    'cost': cost,
                    'reduced_chi_sq': red_chi_sq,
                    'azimuthal_gap': az_gap,
                    'max_uncertainty': max_uncert,
                    'num_stations': len(stations_pos)
                })
        except Exception as e:
            print(f"Erreur pour l'association {i}: {e}")
            continue

    df = pd.DataFrame(quality_data)

    if df.empty:
        print("Aucune donnée de qualité disponible")
        return plt.figure(), pd.DataFrame()

    # Créer une figure avec subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Cost vs Reduced Chi-square
    if 'cost' in df.columns and 'reduced_chi_sq' in df.columns:
        valid_data = df[(df['reduced_chi_sq'] != float('inf')) & (df['reduced_chi_sq'] < 100)]
        if not valid_data.empty:
            axes[0,0].scatter(valid_data['cost'], valid_data['reduced_chi_sq'], alpha=0.6)
            axes[0,0].set_xlabel('Cost')
            axes[0,0].set_ylabel('Reduced Chi-square')
            axes[0,0].set_title('Cost vs Reduced Chi-square')
            axes[0,0].grid(True, alpha=0.3)

    # Gap azimutal vs Nombre de stations
    if 'num_stations' in df.columns and 'azimuthal_gap' in df.columns:
        axes[0,1].scatter(df['num_stations'], df['azimuthal_gap'], alpha=0.6)
        axes[0,1].set_xlabel('Nombre de stations')
        axes[0,1].set_ylabel('Gap azimutal (°)')
        axes[0,1].set_title('Gap azimutal vs Nombre de stations')
        axes[0,1].grid(True, alpha=0.3)

    # Incertitude vs Cost
    if 'max_uncertainty' in df.columns and 'cost' in df.columns:
        df_clean = df.dropna(subset=['max_uncertainty'])
        df_clean = df_clean[df_clean['max_uncertainty'] != float('inf')]
        if not df_clean.empty:
            axes[1,0].scatter(df_clean['cost'], df_clean['max_uncertainty'], alpha=0.6)
            axes[1,0].set_xlabel('Cost')
            axes[1,0].set_ylabel('Incertitude max position')
            axes[1,0].set_title('Incertitude vs Cost')
            axes[1,0].grid(True, alpha=0.3)
        else:
            axes[1,0].text(0.5, 0.5, 'Pas de données d\'incertitude disponibles',
                          ha='center', va='center', transform=axes[1,0].transAxes)

    # Histogramme du gap azimutal
    if 'azimuthal_gap' in df.columns:
        axes[1,1].hist(df['azimuthal_gap'], bins=30, alpha=0.7, edgecolor='black')
        axes[1,1].axvline(180, color='red', linestyle='--', label='Seuil critique (180°)')
        axes[1,1].set_xlabel('Gap azimutal (°)')
        axes[1,1].set_ylabel('Nombre d\'associations')
        axes[1,1].set_title('Distribution du gap azimutal')
        axes[1,1].legend()

    plt.tight_layout()
    return fig, df

def create_interactive_map(results, stations_dict):
    """
    Carte interactive avec Plotly
    """
    # Préparer les données
    event_data = []

    for i, res in results.items():
        if hasattr(res, 'x') and len(res.x) >= 2:
            if len(res.x) == 3:
                lat, lon = res.x[1], res.x[2]
            else:
                lat, lon = res.x[0], res.x[1]

            cost = res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)

            event_data.append({
                'lat': lat,
                'lon': lon,
                'cost': cost,
                'association_id': i,
                'type': 'Événement'
            })

    # Ajouter les stations
    station_data = []
    if stations_dict:
        for idx, station in enumerate(stations_dict):
            pos = station.get_pos()
            station_data.append({
                'lat': pos[0],
                'lon': pos[1],
                'station_id': idx,
                'type': 'Station'
            })

    # Créer la figure Plotly
    fig = go.Figure()

    # Ajouter les événements
    if event_data:
        events_df = pd.DataFrame(event_data)
        fig.add_trace(go.Scatter(
            x=events_df['lon'],
            y=events_df['lat'],
            mode='markers',
            marker=dict(
                size=8,
                color=events_df['cost'],
                colorscale='Viridis_r',
                colorbar=dict(title="Cost"),
                showscale=True
            ),
            text=events_df.apply(lambda row: f"Association {row['association_id']}<br>Cost: {row['cost']:.1f}", axis=1),
            hovertemplate='%{text}<br>Lat: %{y:.4f}<br>Lon: %{x:.4f}<extra></extra>',
            name='Événements'
        ))

    # Ajouter les stations
    if station_data:
        stations_df = pd.DataFrame(station_data)
        fig.add_trace(go.Scatter(
            x=stations_df['lon'],
            y=stations_df['lat'],
            mode='markers',
            marker=dict(
                size=12,
                color='red',
                symbol='triangle-up'
            ),
            text=stations_df.apply(lambda row: f"Station {row['station_id']}", axis=1),
            hovertemplate='%{text}<br>Lat: %{y:.4f}<br>Lon: %{x:.4f}<extra></extra>',
            name='Stations'
        ))

    fig.update_layout(
        title="Carte interactive des associations",
        xaxis_title="Longitude",
        yaxis_title="Latitude",
        showlegend=True,
        height=600
    )

    return fig

def generate_summary_report(results, association_data, stations_dict):
    """
    Génère un rapport de synthèse
    """
    print("="*60)
    print("RAPPORT DE SYNTHÈSE DES ASSOCIATIONS")
    print("="*60)

    # Statistiques générales
    print(f"\n📊 STATISTIQUES GÉNÉRALES")
    print(f"   • Nombre total d'associations: {len(results)}")

    # Analyse des costs
    costs = [res.cost if hasattr(res, 'cost') else np.sum(res.fun**2) for res in results.values()]
    print(f"\n💰 ANALYSE DES COSTS")
    print(f"   • Cost moyen: {np.mean(costs):.2f}")
    print(f"   • Cost médian: {np.median(costs):.2f}")
    print(f"   • Cost min/max: {np.min(costs):.2f} / {np.max(costs):.2f}")
    print(f"   • Écart-type: {np.std(costs):.2f}")

    # Seuils de qualité
    good_threshold = np.percentile(costs, 75)
    excellent_threshold = np.percentile(costs, 25)

    excellent = sum(1 for c in costs if c <= excellent_threshold)
    good = sum(1 for c in costs if excellent_threshold < c <= good_threshold)
    poor = sum(1 for c in costs if c > good_threshold)

    print(f"\n⭐ QUALITÉ DES ASSOCIATIONS")
    print(f"   • Excellentes (Q1): {excellent} ({excellent/len(costs)*100:.1f}%)")
    print(f"   • Bonnes (Q2-Q3): {good} ({good/len(costs)*100:.1f}%)")
    print(f"   • Moyennes (Q4): {poor} ({poor/len(costs)*100:.1f}%)")

    # Analyse des reduced chi-square
    red_chi_sqs = []
    for res in results.values():
        n_obs = len(res.fun)
        n_params = 2
        dof = n_obs - n_params
        red_chi_sq = (res.cost if hasattr(res, 'cost') else np.sum(res.fun**2)) / dof if dof > 0 else float('inf')
        if red_chi_sq != float('inf'):
            red_chi_sqs.append(red_chi_sq)

    if red_chi_sqs:
        print(f"\n📈 REDUCED CHI-SQUARE")
        print(f"   • Médian: {np.median(red_chi_sqs):.2f}")
        print(f"   • < 1.5 (bon ajustement): {sum(1 for x in red_chi_sqs if x < 1.5)} ({sum(1 for x in red_chi_sqs if x < 1.5)/len(red_chi_sqs)*100:.1f}%)")
        print(f"   • > 3.0 (mauvais ajustement): {sum(1 for x in red_chi_sqs if x > 3.0)} ({sum(1 for x in red_chi_sqs if x > 3.0)/len(red_chi_sqs)*100:.1f}%)")

# Fonction principale de visualisation
def visualize_association_results(results, association_data, stations_dict, idx_to_det,
                                 save_plots=True, output_dir="./plots/"):
    """
    Lance toutes les visualisations
    """

    if save_plots:
        os.makedirs(output_dir, exist_ok=True)

    print("Génération des visualisations...")

    # 1. Rapport de synthèse
    generate_summary_report(results, association_data, stations_dict)

    # 2. Carte des associations
    fig1 = plot_association_map(results, stations_dict)
    if save_plots:
        fig1.savefig(f"{output_dir}/association_map.png", dpi=300, bbox_inches='tight')

    # 3. Distribution des costs
    fig2, costs, red_chi_sqs = plot_cost_distribution(results)
    if save_plots:
        fig2.savefig(f"{output_dir}/cost_distribution.png", dpi=300, bbox_inches='tight')

    # 4. Analyse des résidus
    fig3 = plot_residuals_analysis(results, association_data, idx_to_det, stations_dict)
    if save_plots:
        fig3.savefig(f"{output_dir}/residuals_analysis.png", dpi=300, bbox_inches='tight')

    # 5. Métriques de qualité
    fig4, quality_df = plot_quality_metrics(results, association_data, stations_dict)
    if save_plots:
        fig4.savefig(f"{output_dir}/quality_metrics.png", dpi=300, bbox_inches='tight')

    # # 6. Carte interactive (Plotly)
    # fig_interactive = create_interactive_map(results, stations_dict)
    # if True:
    #     fig_interactive.write_html(f"{output_dir}/interactive_map.html")
    #
    # plt.show()

    return {
        'costs': costs,
        'red_chi_squares': red_chi_sqs,
        'quality_dataframe': quality_df,
        # 'interactive_map': fig_interactive
    }

# Utilisation
if __name__ == "__main__":
    # Supposons que vous avez vos résultats filtrés
    # results = {...}  # Votre dictionnaire de résultats

    # Lancer toutes les visualisations
    viz_results = visualize_association_results(
        results=final_results,
        association_data=association,
        stations_dict=STATIONS,
        idx_to_det=IDX_TO_DET,
        save_plots=False,
        output_dir="./association_plots/"
    )

### Checking results

In [ ]:
def analyze_residuals(result):
    """Analyse la qualité des résidus pour évaluer la pertinence de l'association"""
    residuals = result.fun
    num_params = len(result.x) if hasattr(result, 'x') else 0
    num_obs = len(residuals)
    dof = max(1, num_obs - num_params-1)

    if num_obs == 0:
        print("error")

    # Chi-carré réduit
    chi2_reduced = np.sum(residuals**2) / dof

    # Z-score des résidus (résidus standardisés)
    residual_std = np.std(residuals) if np.std(residuals) > 1e-10 else 1e-10
    z_scores = np.abs(residuals) / residual_std
    max_zscore = np.max(z_scores)

    # Fraction d'outliers (|z-score| > 2.5)
    outlier_fraction = np.mean(z_scores > 2.0)

    # Test de normalité des résidus (Shapiro-Wilk si n < 5000, sinon Kolmogorov-Smirnov)
    if num_obs >= 3 and num_obs < 5000:
         _, normality_pvalue = stats.shapiro(residuals)

    # Score de qualité composite (plus c'est haut, mieux c'est)
    quality_score = 1.0 / (1.0 + chi2_reduced)  # Pénalise chi2 élevé
    quality_score *= (1.0 - outlier_fraction)    # Pénalise les outliers
    quality_score *= 1.0 / (1.0 + max_zscore/10.0)  # Pénalise les résidus extrêmes

    return {
        'chi2_reduced': float(chi2_reduced),
        'max_residual_zscore': float(max_zscore),
        'outlier_fraction': float(outlier_fraction),
        'normality_pvalue': float(normality_pvalue),
        'quality_score': float(quality_score),
        'cost':float(res.cost)
    }

In [ ]:
quality = []
rows = []
for i in final_results:
    res = final_results[i]
    rows.append({
        "lat": res.x[1],
        "lon": res.x[2],
        "origin_time": res.x[0],
        "cost": res.cost
    })
    # quality.append(analyze_residuals(res))

# quality = pd.DataFrame(quality)
df = pd.DataFrame(rows)
df["origin_time"] = pd.to_datetime(df["origin_time"], unit="s",utc=True)

try :
    ground_truth['origin_time'] = pd.to_datetime(ground_truth['origin_time'])
    ground_truth['origin_time'] = ground_truth['origin_time'].dt.tz_localize('UTC')
except :
    pass

matches = []

time_threshold = pd.Timedelta(seconds=3*60)
latlon_threshold = 1 # en degrés (à adapter !)

for i, row_res in df.iterrows():
    for j, row_gt in ground_truth.iterrows():
        # condition temporelle
        time_close = abs(row_res["origin_time"] - row_gt["origin_time"]) <= time_threshold

        # condition spatiale (ici simple différence en degrés)
        lat_close = abs(row_res["lat"] - row_gt["lat"]) <= latlon_threshold
        lon_close = abs(row_res["lon"] - row_gt["lon"]) <= latlon_threshold

        if time_close and lat_close and lon_close:
            matches.append((i, j))

print(len(matches))

In [ ]:
def evaluate_precision_recall(df, ground_truth, matches):
    """
    Évalue la précision et le rappel à partir des matches trouvés.

    Args:
        df: DataFrame des détections prédites
        ground_truth: DataFrame de la vérité terrain
        matches: Liste de tuples (index_pred, index_gt) des correspondances

    Returns:
        dict: Métriques d'évaluation
    """
    # Convertir les matches en sets pour éviter les doublons
    matched_pred_indices = set(i for i, j in matches)
    matched_gt_indices = set(j for i, j in matches)

    # Calcul des métriques de base
    true_positives = len(matches)
    false_positives = len(df) - len(matched_pred_indices)
    false_negatives = len(ground_truth) - len(matched_gt_indices)

    # Calcul de Precision et Recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    # F1-score (moyenne harmonique)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'true_positives': true_positives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'n_predictions': len(df),
        'n_ground_truth': len(ground_truth)
    }


In [ ]:
def find_unique_matches(df, ground_truth, time_threshold=pd.Timedelta(seconds=60),
                       latlon_threshold=1):
    """
    Trouve des matches uniques (1 vérité terrain → 1 prédiction maximum)
    """
    matches = []
    used_gt_indices = set()
    used_pred_indices = set()

    # Créer toutes les paires candidates
    candidates = []

    for i, row_res in df.iterrows():
        for j, row_gt in ground_truth.iterrows():
            time_close = abs(row_res["origin_time"] - row_gt["origin_time"]) <= time_threshold
            lat_close = abs(row_res["lat"] - row_gt["lat"]) <= latlon_threshold
            lon_close = abs(row_res["lon"] - row_gt["lon"]) <= latlon_threshold

            if time_close and lat_close and lon_close:
                # Calculer la distance composite comme score de qualité
                time_diff = abs(row_res["origin_time"] - row_gt["origin_time"]).total_seconds()
                spatial_diff = np.sqrt((row_res["lat"] - row_gt["lat"])**2 +
                                     (row_res["lon"] - row_gt["lon"])**2)
                score = time_diff/time_threshold.total_seconds() + spatial_diff/latlon_threshold

                candidates.append((score, i, j))

    # Trier par meilleur score (plus proche)
    candidates.sort(key=lambda x: x[0])

    # Assigner les matches sans doublons
    for score, pred_idx, gt_idx in candidates:
        if pred_idx not in used_pred_indices and gt_idx not in used_gt_indices:
            matches.append((pred_idx, gt_idx))
            used_pred_indices.add(pred_idx)
            used_gt_indices.add(gt_idx)

    return matches

def find_missing_events(df, ground_truth, matches):
    """
    Identifie tous les événements de la vérité terrain qui n'ont pas été détectés
    """
    # Récupérer tous les indices GT qui ont été matchés
    matched_gt_indices = set(j for i, j in matches)

    # Trouver les indices GT qui n'ont PAS été matchés
    missing_gt_indices = set(ground_truth.index) - matched_gt_indices

    if len(missing_gt_indices) == 0:
        print("Aucun événement manquant trouvé.")
        return pd.DataFrame()

    print(f"ÉVÉNEMENTS RÉELS MANQUANTS: {len(missing_gt_indices)}")
    missing_events = []

    for gt_idx in missing_gt_indices:
        missing_event = ground_truth.loc[gt_idx]
        missing_events.append(missing_event)
        print(f"\n{'='*60}")
        print(f"ÉVÉNEMENT MANQUANT {len(missing_events)+1}/{len(missing_gt_indices)} (Index {gt_idx})")
        print(f"{'='*60}")
        print(f"Time: {missing_event['origin_time']}")
        print(f"Latitude: {missing_event['lat']:.4f}")
        print(f"Longitude: {missing_event['lon']:.4f}")
    missing_events = pd.DataFrame(missing_events)
    return pd.DataFrame(missing_events)

def find_false_positives(df, ground_truth, matches):

    # Récupérer tous les indices df qui ont été matchés
    matched_indices = set(i for i, j in matches)
    # Trouver les indices df qui n'ont PAS été matchés
    missing_indices = set(df.index) - matched_indices
    if len(missing_indices) == 0:
        print("Aucun événement manquant trouvé.")
        return pd.DataFrame()

    print(f"False Positives (FP): {len(missing_indices)}")
    false_positives = []

    for idx in missing_indices:
        missing_event = df.loc[idx]
        false_positives.append(missing_event)
        # print(f"{'='*60}")
        # print(f"False Positives (FP) {len(false_positives)+1}/{len(missing_indices)} (Index {idx})")
        # # print(f"{'='*60}")
        # print(f"Time: {missing_event['origin_time']}")
        # print(f"Latitude: {missing_event['lat']:.4f}")
        # print(f"Longitude: {missing_event['lon']:.4f}")
    false_positives = pd.DataFrame(false_positives)
    return false_positives

# Réévaluation avec matching unique
missing_events = find_missing_events(df, ground_truth, matches)
find_false_positives(df, ground_truth, matches)
unique_matches = find_unique_matches(df, ground_truth)
metrics = evaluate_precision_recall(df, ground_truth, unique_matches)
false_positives= find_false_positives(df, ground_truth, unique_matches)
print("=== ÉVALUATION CORRIGÉE ===")
print(f"Prédictions trouvées: {metrics['n_predictions']}")
print(f"Événements réels: {metrics['n_ground_truth']}")
print(f"True Positives (TP): {metrics['true_positives']}")
print(f"False Positives (FP): {metrics['false_positives']}")
print(f"False Negatives (FN): {metrics['false_negatives']}")
print(f"Précision: {metrics['precision']:.3f} ({metrics['precision']*100:.1f}%)")
print(f"Rappel: {metrics['recall']:.3f} ({metrics['recall']*100:.1f}%)")
print(f"F1-Score: {metrics['f1_score']:.3f}")

In [ ]:
print("=== ÉVALUATION CORRIGÉE ===")
print(f"Prédictions trouvées: {metrics['n_predictions']}")
print(f"Événements réels: {metrics['n_ground_truth']}")
print(f"True Positives (TP): {metrics['true_positives']}")
print(f"False Positives (FP): {metrics['false_positives']}")
print(f"False Negatives (FN): {metrics['false_negatives']}")
print(f"Précision: {metrics['precision']:.3f} ({metrics['precision']*100:.1f}%)")
print(f"Rappel: {metrics['recall']:.3f} ({metrics['recall']*100:.1f}%)")
print(f"F1-Score: {metrics['f1_score']:.3f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Carte simple des événements
plt.figure(figsize=(12, 10))

# True Positives (TP) - en bleu
tp_indices = [i for i, j in unique_matches]
tp_points = df.loc[tp_indices]
plt.scatter(tp_points['lon'], tp_points['lat'],
           c='blue', marker='o', s=50, alpha=0.7, label=f'TP ({len(tp_points)})')

# False Positives (FP) - en orange
if not false_positives.empty:
    plt.scatter(false_positives['lon'], false_positives['lat'],
               c='orange', marker='x', s=50, alpha=0.7, label=f'FP ({len(false_positives)})')

# Ground Truth (GT) - contours verts
plt.scatter(ground_truth['lon'], ground_truth['lat'],
           facecolors='none', edgecolors='green', marker='s',
           s=80, linewidth=2, label=f'GT ({len(ground_truth)})')

# Missing Events (FN) - en rouge
if not missing_events.empty:
    plt.scatter(missing_events['lon'], missing_events['lat'],
               c='red', marker='+', s=80, alpha=0.8, label=f'FN ({len(missing_events)})')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Carte des Événements - TP, FP, GT, FN')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 2. Histogramme des coûts pour TP et FP
plt.figure(figsize=(12, 6))

# Extraire les coûts
tp_costs = tp_points['cost'].values
fp_costs = false_positives['cost'].values if not false_positives.empty else []

# Créer l'histogramme comparatif
if len(fp_costs) > 0:
    # Histogramme comparatif TP vs FP
    bins = np.linspace(min(np.min(tp_costs), np.min(fp_costs)),
                      max(np.max(tp_costs), np.max(fp_costs)), 30)

    plt.hist(tp_costs, bins=bins, alpha=0.7, color='blue', label=f'TP ({len(tp_costs)})', density=True)
    plt.hist(fp_costs, bins=bins, alpha=0.7, color='orange', label=f'FP ({len(fp_costs)})', density=True)
else:
    # Seulement TP si pas de FP
    plt.hist(tp_costs, bins=30, alpha=0.7, color='blue', label=f'TP ({len(tp_costs)})', density=True)

plt.xlabel('Coût')
plt.ylabel('Densité')
plt.title('Distribution des Coûts - TP vs FP')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Statistiques descriptives des coûts
print("\n=== STATISTIQUES DES COÛTS ===")
print(f"TP - Moyenne: {np.mean(tp_costs):.3f}, Médiane: {np.median(tp_costs):.3f}, Std: {np.std(tp_costs):.3f}")

if len(fp_costs) > 0:
    print(f"FP - Moyenne: {np.mean(fp_costs):.3f}, Médiane: {np.median(fp_costs):.3f}, Std: {np.std(fp_costs):.3f}")

    # Test statistique simple
    if np.mean(fp_costs) > np.mean(tp_costs):
        print("→ Les FP ont tendance à avoir des coûts plus élevés")
    else:
        print("→ Les TP ont tendance à avoir des coûts plus élevés")

# 4. Boxplot comparatif des coûts
if len(fp_costs) > 0:
    plt.figure(figsize=(10, 6))
    data = [tp_costs, fp_costs]
    labels = ['TP', 'FP']

    plt.boxplot(data, labels=labels, patch_artist=True,
               boxprops=dict(facecolor='lightblue', color='blue'),
               medianprops=dict(color='red'))

    plt.ylabel('Coût')
    plt.title('Boxplot des Coûts - TP vs FP')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
find_missing_events(df, ground_truth, matches)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Configuration minimale
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 1. Carte simple
tp_indices = [i for i, j in unique_matches]
tp_points = df.loc[tp_indices]

# Points sur la carte
ax1.scatter(tp_points['lon'], tp_points['lat'], c='blue', label=f'TP ({len(tp_points)})', alpha=0.7)
if not false_positives.empty:
    ax1.scatter(false_positives['lon'], false_positives['lat'], c='orange', marker='x', label=f'FP ({len(false_positives)})')
ax1.scatter(ground_truth['lon'], ground_truth['lat'], facecolors='none', edgecolors='green', label='GT')
if not missing_events.empty:
    ax1.scatter(missing_events['lon'], missing_events['lat'], c='red', marker='^', label=f'FN ({len(missing_events)})')

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Carte des Événements')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Histogramme des coûts
tp_costs = tp_points['cost'].values
fp_costs = false_positives['cost'].values if not false_positives.empty else []

if len(fp_costs) > 0:
    # ax2.hist(tp_costs, bins=20, alpha=0.7,  color='blue', label='TP')
    ax2.hist(fp_costs, bins=20, alpha=0.7, color='orange', label='FP')
else:
    ax2.hist(tp_costs, bins=20, alpha=0.7, color='blue', label='TP', density=True)

ax2.set_xlabel('Coût')
ax2.set_ylabel('Densité')
ax2.set_title('Distribution des Coûts')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Affichage des stats en une ligne
print(f"Coût TP: {np.mean(tp_costs):.3f} ± {np.std(tp_costs):.3f}")
if len(fp_costs) > 0:
    print(f"Coût FP: {np.mean(fp_costs):.3f} ± {np.std(fp_costs):.3f}")

In [ ]:
plt.hist(tp_costs, bins=3, alpha=0.7, color='orange', label='TP')

In [ ]:
# Analyse du seuil de coût optimal pour séparer TP et FP
if len(fp_costs) > 0:
    from sklearn.metrics import roc_curve, auc

    # Préparer les données pour la courbe ROC
    y_true = np.concatenate([np.ones(len(tp_costs)), np.zeros(len(fp_costs))])
    y_scores = np.concatenate([tp_costs, fp_costs])

    # Inverser les coûts (car généralement, coût bas = bon)
    y_scores_inv = -y_scores  # ou utiliser y_scores_max = np.max(y_scores) - y_scores

    fpr, tpr, thresholds = roc_curve(y_true, y_scores_inv)
    roc_auc = auc(fpr, tpr)

    # Trouver le seuil optimal (plus proche du coin en haut à gauche)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = -thresholds[optimal_idx]  # re-inverser

    plt.figure(figsize=(10, 5))

    # Courbe ROC
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=100,
                label=f'Seuil optimal: {optimal_threshold:.3f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Courbe ROC basée sur le coût')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Distribution avec seuil optimal
    plt.subplot(1, 2, 2)
    plt.hist(tp_costs, bins=20, alpha=0.7, color='blue', label='TP', density=True)
    plt.hist(fp_costs, bins=20, alpha=0.7, color='orange', label='FP', density=True)
    plt.axvline(optimal_threshold, color='red', linestyle='--', linewidth=2,
                label=f'Seuil optimal: {optimal_threshold:.3f}')
    plt.xlabel('Coût')
    plt.ylabel('Densité')
    plt.title('Seuil de coût optimal')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"Seuil de coût optimal: {optimal_threshold:.3f}")
    print(f"TP en dessous du seuil: {np.sum(tp_costs <= optimal_threshold)}/{len(tp_costs)}")
    print(f"FP au-dessus du seuil: {np.sum(fp_costs > optimal_threshold)}/{len(fp_costs)}")

### Find best quality score

In [ ]:
def analyze_residuals(result):
    """Analyse la qualité des résidus pour évaluer la pertinence de l'association"""
    residuals = result.fun
    num_obs = len(residuals)
    chi2_stat, p_value, passes, dof = SOUND_MODEL.test_chi_square(result)
    if num_obs == 0:
        print("error")
    # Chi-carré réduit
    chi2_reduced = np.sum(residuals**2) / dof
    # Z-scroe des résidus (résidus standardisés)
    residual_std = np.std(residuals) if np.std(residuals) > 1e-10 else 1e-10
    z_scores = np.abs(residuals) / residual_std
    max_zscore = np.max(z_scores)
    outliers = SOUND_MODEL.detect_outliers(result,method='absolute')
    outliers_score = len(outliers['outlier_indices'])
    # Fraction d'outliers (|z-score| > 2.5)
    outlier_fraction = np.mean(z_scores >1.5)

    # Test unitaire de variance
    p = 2  # nombre de paramètres ajustés (ici a et b)
    sigma_theorique = 3
    s2 = np.var(residuals, ddof=p)  # variance sans biais
    # print(f"Variance empirique : {s2:.3f}")
    # Test de normalité des résidus (Shapiro-Wilk si n < 5000, sinon Kolmogorov-Smirnov)
    if num_obs >= 3 and num_obs < 5000:
         _, normality_pvalue = stats.shapiro(residuals)
    # Score de qualité composite (plus c'est haut, mieux c'est)
    quality_score = 1.0 / (1.0 + chi2_reduced)  # Pénalise chi2 élevé
    quality_score *= (1.0 - outlier_fraction)    # Pénalise les outliers
    quality_score *= 1.0 / (1.0 + max_zscore/10.0)  # Pénalise les résidus extrêmes
    return {
        'chi2_reduced': float(chi2_reduced),
        'max_residual_zscore': float(max_zscore),
        'outlier_fraction': float(outlier_fraction),
        'normality_pvalue': float(normality_pvalue),
        'quality_score': float(quality_score),
        'cost':float(result.cost),
        'chi2_stat': float(chi2_stat),
        'p_value': float(p_value),
        'passes': bool(passes),
        "outliers_score" : outliers_score
    }

In [ ]:
rows = []
for i in final_results:
    res = final_results[i]
    metrics = analyze_residuals(res)  # ton analyse des résidus

    rows.append({
        "lat": res.x[1],
        "lon": res.x[2],
        "origin_time": res.x[0],
        "cost": res.cost,
        "chi2_reduced": metrics["chi2_reduced"],
        "outlier_fraction": metrics["outlier_fraction"],
        "max_zscore": metrics["max_residual_zscore"],
        "normality_pvalue": metrics["normality_pvalue"],
        "quality_score": metrics["quality_score"],
        'chi2_stat' : metrics["chi2_stat"],
        'p_value': metrics["p_value"],
        'passes': metrics["passes"],
        "outliers_score" : metrics["outliers_score"],
    })

df = pd.DataFrame(rows)
df["origin_time"] = pd.to_datetime(df["origin_time"], unit="s", utc=True)


df = pd.DataFrame(rows)
df["origin_time"] = pd.to_datetime(df["origin_time"], unit="s",utc=True)

try :
    ground_truth['origin_time'] = pd.to_datetime(ground_truth['origin_time'])
    ground_truth['origin_time'] = ground_truth['origin_time'].dt.tz_localize('UTC')
except :
        ground_truth['origin_time'] = pd.to_datetime(ground_truth['origin_time'])
        ground_truth['origin_time'] = ground_truth['origin_time'].dt.tz_convert('UTC')

matches = []

time_threshold = pd.Timedelta(seconds=1*60)
latlon_threshold = 1 # en degrés (à adapter !)
df["is_match"] = False
for i, row_res in df.iterrows():
    for j, row_gt in ground_truth.iterrows():
        # condition temporelle
        time_close = abs(row_res["origin_time"] - row_gt["origin_time"]) <= time_threshold

        # condition spatiale (ici simple différence en degrés)
        lat_close = abs(row_res["lat"] - row_gt["lat"]) <= latlon_threshold
        lon_close = abs(row_res["lon"] - row_gt["lon"]) <= latlon_threshold

        if time_close and lat_close and lon_close:
            matches.append((i, j))
            df.at[i, "is_match"] = True

print(len(matches))

In [ ]:
['cost', 'chi2_reduced', 'outlier_fraction','max_zscore', 'normality_pvalue', 'quality_score', 'chi2_stat','p_value', 'passes', 'outliers_score', 'is_match']

In [ ]:
from sklearn.metrics import roc_auc_score

benchmark = {}

for col in ['cost', 'chi2_reduced', 'outlier_fraction','max_zscore', 'normality_pvalue', 'quality_score', 'chi2_stat','p_value', 'passes', 'outliers_score', 'is_match']:
    try:
        auc = roc_auc_score(df["is_match"], -df[col] if col in ["cost","chi2_reduced", "outlier_fraction", "max_zscore",'chi2_stat','outliers_score'] else df[col])
        benchmark[col] = auc
        benchmark[col] = auc
    except ValueError as e:
        print("error", e)
        benchmark[col] = None

print(benchmark)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for col in ['cost', 'chi2_reduced', 'outlier_fraction','max_zscore', 'normality_pvalue', 'quality_score', 'chi2_stat','p_value', 'passes', 'outliers_score']:
    plt.figure()
    sns.histplot(data=df, x=col, hue="is_match", kde=True, stat="density", common_norm=False)
    plt.title(col)
    plt.show()

In [ ]:
final_results